<a href="https://colab.research.google.com/github/AIManifest/ComfyUI-Notebook/blob/master/ComfyUI-Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#<left><font face="Monospace" size="8"><b>ComfyUI-Notebook</b></font></left>

#<left><font face="Monospace" size="6">Initialize Environment and Models</font></left>


In [ ]:
#@title <left><font face="Monospace" size="4">Step 1. Setup HOME Path for Root Working Directory (Required)</font></left>
try:
    from google.colab import drive
    gdrive = '/content/drive/'
    drive.mount(gdrive)
    HOME = '/content/'
    print(f'Root Working Directory: {HOME}')
except:
    print(f'Colab not Detected, Changing Paths to Appropriate Root Directory ("/workspace") for Jupyter')
    HOME = '/workspace/'
    print(f'Root Working Directory: {HOME}')

In [ ]:
#@title <left><font face="Monospace" size="4">Step 2: Run This Cell To Setup Jupyter Notebook/Lab to Run ComfyUI-Notebook</font></left>
import os
import subprocess

override_working_dir = False #@param{type:"boolean"}
override_dir = "/content/ComfyUI-Notebook/" #@param{type:"string"}
run_full_setup_for_jupyter = False #@param{type:"boolean"}
install_comfy_notebook = True #@param{type:"boolean"}
install_comfy_ui = False #@param{type:"boolean"}
install_esrgan = False #@param{type:"boolean"}
xformers_version = 'xformers==0.0.22.dev591' #@param{type:"string"}

def setup_comfy():
    if run_full_setup_for_jupyter:
        subprocess.run('curl -fsSL https://deb.nodesource.com/setup_18.x | sudo -E bash -', shell=True)
        subprocess.run(['sudo', 'apt-get', 'install', '-y', 'nodejs'])

        subprocess.run(['apt-get', 'update'])
        subprocess.run(['apt-get', 'install', 'ffmpeg', 'libsm6', 'libxext6', '-y'])
        subprocess.run(['pip', 'install', 'pandas'])

        subprocess.run(['sudo', 'apt-get', 'update'])
        subprocess.run(['sudo', 'apt-get', 'install', 'npm', '-y'])
        subprocess.run(['conda', 'install', '-c', 'nvidia', 'cuda-toolkit', '-y'])
    if install_comfy_notebook:
        os.system("git clone https://github.com/AIManifest/ComfyUI-Notebook.git")
    if install_comfy_ui:
        os.system("git clone https://github.com/comfyanonymous/ComfyUI.git")
    if install_esrgan:
        os.system("git clone https://github.com/AIManifest/ESRGAN.git")

    WORKING_DIR = os.path.join(HOME, "ComfyUI-Notebook/")
    if override_working_dir:
        WORKING_DIR = override_dir
    os.chdir(WORKING_DIR)
    print(f'Running in Directory: {WORKING_DIR}')
    subprocess.run(['pip', 'install', '-r', 'requirements.txt'])
    subprocess.run(['pip', 'install', str(xformers_version), 'pytorch-lightning', 'flash-attn', '--no-build-isolation'])
    os.system('cd custom_nodes && git clone https://github.com/Fannovel16/comfy_controlnet_preprocessors; cd comfy_controlnet_preprocessors && python install.py')

setup_comfy()


In [ ]:
#@title <left><font face="Monospace" size="4">Step 3.a: Run This Cell To Copy Models from One Folder to Root Folder</font></left>
source_folder = "/content/drive/MyDrive/ComfyUI/models/checkpoints" #@param{type:"string"}
root_folder = "/content/ComfyUI-Notebook/models" #@param{type:"string"}
!cp -r "$source_folder" "$root_folder" --verbose

In [ ]:
#@title <left><font face="Monospace" size="4">Step 3.b: Install Models</font></left>
import os
import time
import shutil
import requests
from tqdm.auto import tqdm

model_dir = "/content/ComfyUI-Notebook/models/checkpoints/" #@param{type:"string"}
vae_dir = "/content/ComfyUI-Notebook/models/vae/" #@param{type:"string"}
lora_dir = "/content/ComfyUI-Notebook/models/loras/" #@param{type:"string"}
controlnet_dir = "/content/ComfyUI-Notebook/models/controlnet" #@param{type:"string"}

# Add boolean parameters for each model
download_sd_xl_offset_example_lora_1_0 = False #@param {type:"boolean"}
download_sd_xl_base_1_0 = False #@param {type:"boolean"}
download_sd_xl_refiner_1_0 = False #@param {type:"boolean"}
download_sd_xl_base_1_0_0_9vae = False #@param {type:"boolean"}
download_sd_xl_refiner_1_0_0_9vae = True #@param {type:"boolean"}
download_dreamshaperXL10_alpha2XI10 = False #@param {type:"boolean"}
download_psyanimated_xl = False #@param{type:"boolean"}
download_crystalClearxl_ccxl = True #@param{type:"boolean"}
download_canny_sdxl = False #@param{type:"boolean"}
download_depth_sdxl = True #@param{type:"boolean"}
download_sdxl_vae = True #@param{type:"boolean"}
def download_sdxl():
    models_to_download = [
        ("https://huggingface.co/stabilityai/sdxl-vae/resolve/main/sdxl_vae.safetensors", vae_dir, "sdxl_vae.safetensors", download_sdxl_vae),
        ("https://huggingface.co/diffusers/controlnet-depth-sdxl-1.0/resolve/main/diffusion_pytorch_model.safetensors", controlnet_dir, "sdxl_1-0_depth.safetensors", download_depth_sdxl),
        ("https://huggingface.co/diffusers/controlnet-canny-sdxl-1.0/resolve/main/diffusion_pytorch_model.safetensors", controlnet_dir, "sdxl_1-0_canny.safetensors", download_canny_sdxl),
        ("https://civitai.com/api/download/models/133832", model_dir, "crystalClearxl_ccxl.safetensors", download_crystalClearxl_ccxl),
        ("https://civitai.com/api/download/models/136271", model_dir, "psyanimated_xl.safetensors", download_psyanimated_xl),
        ("https://huggingface.co/stabilityai/stable-diffusion-xl-base-1.0/resolve/main/sd_xl_offset_example-lora_1.0.safetensors", lora_dir, "sd_xl_offset_example-lora_1.0.safetensors", download_sd_xl_offset_example_lora_1_0),
        ("https://huggingface.co/stabilityai/stable-diffusion-xl-base-1.0/resolve/main/sd_xl_base_1.0.safetensors", model_dir, "sd_xl_base_1.0.safetensors", download_sd_xl_base_1_0),
        ("https://huggingface.co/stabilityai/stable-diffusion-xl-refiner-1.0/resolve/main/sd_xl_refiner_1.0.safetensors", model_dir, "sd_xl_refiner_1.0.safetensors", download_sd_xl_refiner_1_0),
        ("https://huggingface.co/stabilityai/stable-diffusion-xl-base-1.0/resolve/main/sd_xl_base_1.0_0.9vae.safetensors", model_dir, "sd_xl_base_1.0_0.9vae.safetensors", download_sd_xl_base_1_0_0_9vae),
        ("https://huggingface.co/stabilityai/stable-diffusion-xl-refiner-1.0/resolve/main/sd_xl_refiner_1.0_0.9vae.safetensors", model_dir, "sd_xl_refiner_1.0_0.9vae.safetensors", download_sd_xl_refiner_1_0_0_9vae),
        ("https://civitai.com/api/download/models/126688", model_dir, "dreamshaperXL10_alpha2XI10.safetensors", download_dreamshaperXL10_alpha2XI10)
    ]

    username = "" #@param{type:"string"}
    password = "" #@param{type:"string"}

    for model_URL, folder_path, filename, download in models_to_download:
        if not download:
            continue

        if not os.path.exists(folder_path):
            os.makedirs(folder_path, exist_ok=True)

        if not os.path.exists(os.path.join(folder_path, filename)):
            start_time = time.time()

            try:
                auth = (username, password) if username and password else None
                with requests.get(model_URL, stream=True, auth=auth) as r:

                    total_length = int(r.headers.get("Content-Length"))

                    with tqdm.wrapattr(r.raw, "read", total=total_length, desc="..Downloading..", colour='orange') as raw:

                        with open(os.path.join(folder_path, filename), 'wb')as output:
                            shutil.copyfileobj(raw, output)

                end_time = time.time()
                final_time = end_time-start_time
                print(f"\033[38;2;255;192;203mDone! Model {filename} downloaded in {final_time:.2f} seconds!")
                print(f"\033[38;2;255;192;203mFile {filename} downloaded and saved to {folder_path}")
            except Exception as e:
                print(f"An error occurred while downloading the model: {e}")

download_sdxl()

In [ ]:
#@title <left><font face="Monospace" size="4">Step 4: Set Memory Options</font></left>
import yaml

vram_state = "NORMAL_VRAM" #@param["DISABLED", "NO_VRAM", "LOW_VRAM", "NORMAL_VRAM", "HIGH_VRAM", "SHARED"]
lowvram_available = True #@param{type:"boolean"}
xpu_available = False #@param{type:"boolean"}
directml_enabled = False #@param{type:"boolean"}
directml_device = None #@param{type:"boolean"}
use_cpu = False #@param{type:"boolean"}
gpu_only = False #@param{type:"boolean"}
normalvram = False #@param{type:"boolean"}
use_pytorch_cross_attention = False #@param{type:"boolean"}

use_split_cross_attention = False #@param{type:"boolean"}
use_quad_cross_attention = False #@param{type:"boolean"}
novram = False #@param{type:"boolean"}
lowvram = False #@param{type:"boolean"}
highvram = False #@param{type:"boolean"}
gpu_only = False #@param{type:"boolean"}
force_fp32 = False #@param{type:"boolean"}
force_fp16 = False #@param{type:"boolean"}
disable_xformers = False #@param{type:"boolean"}
fp16_vae = False #@param{type:"boolean"}
bf16_vae = False #@param{type:"boolean"}
XFORMERS_IS_AVAILABLE = True #@param{type:"boolean"}
XFORMERS_ENABLED_VAE = True #@param{type:"boolean"}
cuda_malloc = True #@param{type:"boolean"}
disable_cuda_malloc = False #@param{type:"boolean"}
data = {
    'vram_state': vram_state,
    'lowvram_available': lowvram_available,
    'xpu_available': xpu_available,
    'directml_enabled': directml_enabled,
    'directml_device': directml_device,
    'use_cpu': use_cpu,
    'gpu_only': gpu_only,
    'normalvram': normalvram,
    'use_pytorch_cross_attention': use_pytorch_cross_attention,
    'use_split_cross_attention': use_split_cross_attention,
    'use_quad_cross_attention': use_quad_cross_attention,
    'novram': novram,
    'lowvram': lowvram,
    'highvram': highvram,
    'force_fp32': force_fp32,
    'force_fp16': force_fp16,
    'disable_xformers': disable_xformers,
    'fp16_vae': fp16_vae,
    'bf16_vae': bf16_vae,
    'XFORMERS_IS_AVAILABLE': XFORMERS_IS_AVAILABLE,
    'XFORMERS_ENABLED_VAE': XFORMERS_ENABLED_VAE,
    'cuda_malloc': cuda_malloc,
    'disable_cuda_malloc': disable_cuda_malloc
}

with open('/content/ComfyUI-Notebook/comfy_notebook.yaml', 'w') as file:
    yaml.dump(data, file)


#<left><font face="Monospace" size="6">Run ComfyUI-Notebook For Web Browser</font></left>

In [ ]:
#@title <left><font face="Monospace" size="4">Run ComfyUI-Notebook For Web Browser</font></left>
working_dir = "/content/ComfyUI" #@param{type:"string"}
%cd $working_dir
!npm install -g localtunnel

import subprocess
import threading
import time
import socket
import urllib.request

preview_method = "auto" #@param{type:"string"}["none", "auto", "latent2rgb", "taesd"]

def iframe_thread(port):
  while True:
      time.sleep(0.5)
      sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
      result = sock.connect_ex(('127.0.0.1', port))
      if result == 0:
        break
      sock.close()
  print("\nComfyUI-Notebook finished loading, trying to launch localtunnel (if it gets stuck here localtunnel is having issues)\n")

  print("The password/enpoint ip for localtunnel is:", urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))
  p = subprocess.Popen(["lt", "--port", "{}".format(port)], stdout=subprocess.PIPE)
  for line in p.stdout:
    print(line.decode(), end='')


threading.Thread(target=iframe_thread, daemon=True, args=(8188,)).start()

!python main.py --dont-print-server --preview-method $preview_method



In [ ]:
#@title <left><font face="Monospace" size="4">(Advanced) Run ComfyUI-Notebook For Web Browser</font></left>

#@markdown <left><font face="Monospace" size="4">Open This Cell to Add Your Extra ComfyUI Launch Arguments</font></left>

working_dir = "/content/ComfyUI" #@param{type:"string"}
%cd $working_dir
!npm install -g localtunnel

import subprocess
import threading
import time
import socket
import urllib.request

# preview_method = "auto" #@param{type:"string"}["none", "auto", "latent2rgb", "taesd"]
# listen = "127.0.0.1" #@param{type:"string"}
# port = 8188 #@param{type:"number"}
# enable_cors_header = None #@param{type:"string"}
# extra_model_paths_config = None #@param{type:"string"}
# output_directory = None #@param{type:"string"}
# temp_directory = None #@param{type:"string"}
# auto_launch = False #@param{type:"boolean"}

# disable_auto_launch = False #@param{type:"boolean"}

# cuda_device = 0 #@param{type:"number"}

# cuda_malloc = False #@param{type:"boolean"}
# disable_cuda_malloc = False #@param{type:"boolean"}

# dont_upcast_attention = False #@param{type:"boolean"}
# force_fp32 = False #@param{type:"boolean"}
# force_fp16 = False #@param{type:"boolean"}

# fp16_vae = False #@param{type:"boolean"}
# bf16_vae = False #@param{type:"boolean"}

# directml = None #@param{type:"number"}
# use_split_cross_attention = False #@param{type:"boolean"}
# use_quad_cross_attention = False #@param{type:"boolean"}
# use_pytorch_cross_attention = False #@param{type:"boolean"}
# disable_xformers = False #@param{type:"boolean"}

# gpu_only = False #@param{type:"boolean"}
# highvram = False #@param{type:"boolean"}
# normalvram = False #@param{type:"boolean"}
# lowvram = False #@param{type:"boolean"}
# novram = False #@param{type:"boolean"}
# cpu = False #@param{type:"boolean"}

# dont_print_server = False #@param{type:"boolean"}
# quick_test_for_ci = False #@param{type:"boolean"}
# windows_standalone_build = False #@param{type:"boolean"}

# disable_metadata = False #@param{type:"boolean"}


def iframe_thread(port):
  while True:
      time.sleep(0.5)
      sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
      result = sock.connect_ex(('127.0.0.1', port))
      if result == 0:
        break
      sock.close()
  print("\nComfyUI-Notebook finished loading, trying to launch localtunnel (if it gets stuck here localtunnel is having issues)\n")

  print("The password/enpoint ip for localtunnel is:", urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))
  p = subprocess.Popen(["lt", "--port", "{}".format(port)], stdout=subprocess.PIPE)
  for line in p.stdout:
    print(line.decode(), end='')


threading.Thread(target=iframe_thread, daemon=True, args=(8188,)).start()

!python main.py --dont-print-server \
--preview-method $preview_method \
# --listen $listen \
# --port $port\
# --enable-cors-header
# --extra-model-paths-config
# --output-directory
# --temp-directory
# --auto-launch
# --disable-auto-launch
# --cuda-device
# --cuda-malloc
# --disable-cuda-malloc
# --dont-upcast-attention
# --force-fp32 \
# --force-fp16
# --fp16-vae
# --bf16-vae
# --directml
# --disable-xformers
# --gpu-only
# --highvram
# --normalvram
# --lowvram
# --novram
# --cpu
# --quick-test-for-ci
# --windows-standalone-build
# --disable-metadata

#<left><font face="Monospace" size="6">Run ComfyUI-Notebook System</font></left>

##<left><font face="Monospace" size="6">Run SDXL -> Single Prompt</font></left>


In [ ]:
#@title <left><font face="Monospace" size="4">Set CudaMallocAsync</font></left>
import os
env_var = "backend:cudaMallocAsync"
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = env_var

In [ ]:
#@title <left><font face="Monospace" size="4">RUN SDXL -> Single Prompt!</font></left>
working_dir = "/content/ComfyUI-Notebook" #@param{type:"string"}
%cd $working_dir
import gc
import torch
import random
import nodes
import sdxlrun
import ipywidgets
from types import SimpleNamespace
from comfy import model_management
from sdxlrun import runsdxl, loadsdxl
from pytorch_lightning import seed_everything
from refinersdxlrun import runsdxlrefiner, loadsdxlrefiner
from iprogress import iprogress

def SDXL_ARGS():
    prompt = "(horror art style), (gruesome), intricate concept art of a (mad scientist's laboratory:1.1) with (grotesque experiments:1.2), (flickering lights:1.3), (green colors), (repulsive), high quality, fluorescent light, stomach-churning atmosphere" #@param{type:"string"}
    negativeprompt = "close up image, (blur:1.8), fake, anime, cartoon, girl, lady, female, bad art, blurry, blurry background, bokeh, bad drawing, poor composition, poor use of textures, poor use of colors, poor art drawing, portrait, ugly, face shot, Overbaked, over contrasted, extra fingers, deformed hands, over saturated, ((3d, cartoon, anime, sketches)), (worst quality:2), (low quality:2), (normal quality:2), lowres, normal quality, ((monochrome)), ((grayscale)), bad anatomy, out of view, cut off, ugly, deformed, mutated, ((young)), EasyNegative, paintings, sketches, (worst quality:2), (low quality:2), (normal quality:2), lowres, normal quality, ((monochrome)), ((grayscale)), skin spots, acnes, skin blemishes, age spot, glans,extra fingers,fewer fingers, (ugly eyes, deformed iris, deformed pupils, fused lips and teeth:1.2), (un-detailed skin, semi-realistic, cgi, 3d, render, sketch, cartoon, drawing, anime:1.2), text, close up, cropped, out of frame, worst quality, low quality, jpeg artifacts, ugly, duplicate, morbid, mutilated, extra fingers, mutated hands, poorly drawn hands, poorly drawn face, mutation, deformed, blurry, dehydrated, bad anatomy, bad proportions, extra limbs, cloned face, disfigured, gross proportions, malformed limbs, missing arms, missing legs, extra arms, extra legs, fused fingers, too many fingers, long neck," #@param{type:"string"}
    #@markdown ${\textsf{Base Model Options}}$
    ckpt_path='/content/ComfyUI/models/checkpoints/dreamshaperXL10_alpha2XI10.safetensors'
    ckpt_name='crystalClearxl_ccxl.safetensors' #@param{type:"string"}["crystalClearxl_ccxl.safetensors", "psyanimated_xl.safetensors", "sd_xl_base_1.0_0.9vae.safetensors", "dreamshaperXL10_alpha2XI10.safetensors"]
    lora_name = "None"#@param{type:"string"}["None", "sd_xl_offset_example-lora_1.0.safetensors", "greg_rutkowski_xl_2.safetensors"]
    if lora_name == "None":
        lora_name = None
    else:
        strength_model = 0.5 #@param{type:"number"}
        strength_clip = 0.5 #@param{type:"number"}

    #@markdown ${\textsf{Base Model}}$
    steps = 35 #@param{type:"number"}
    start_step = 0 #@param{type:"number"}
    last_step = 30 #@param{type:"number"}

    #refiner model
    use_refiner =  False #@param{type:"boolean"}
    last_step = last_step if use_refiner else steps
    refiner_steps = steps
    refiner_start_step = last_step
    refiner_last_step = steps

    #@markdown ${\textsf{VAE Settings}}$
    vae_path = "/content/ComfyUI-Notebook/models/vae/sdxl_vae.safetensors" #@param{type:"string"}
    vae_decode_method = "tiled" #@param{type:"string"}["normal", "tiled"]

    #@markdown ${\textsf{Image-Batch Settings}}$
    imagewidth = 1024 #@param{type:"number"}
    imageheight = 1344   #@param{type:"number"}
    batch_size = 1 #@param{type:"number"}

    cfg = 3.5 #@param{type:"number"}
    seed = -1 #@param{type:"number"}
    if seed == -1:
        seed = random.randint(1, 432043209)
    seed = seed_everything(seed)

    #@markdown ${\textsf{Sampler Settings}}$
    sampler_name = "dpmpp_2m_sde_gpu" #@param{type:"string"} ["euler", "euler_ancestral", "heun", "dpm_2", "dpm_2_ancestral", "lms", "dpm_fast", "dpm_adaptive", "dpmpp_2s_ancestral", "dpmpp_sde", "dpmpp_sde_gpu", "dpmpp_2m", "dpmpp_2m_sde", "dpmpp_2m_sde_gpu", "ddim", "uni_pc", "uni_pc_bh2"]
    scheduler = "exponential" #@param{type:"string"} ["normal", "karras", "exponential", "simple", "ddim_uniform"]

    #@markdown ${\textsf{Sampler Options}}$
    s_churn = 0.01 # @param {type:"slider", min:0, max:1, step:0.01}
    s_tmin = 0.01 # @param {type:"slider", min:0, max:1, step:0.01}
    s_tmax = float('inf')
    s_noise = 1. # @param {type:"slider", min:0, max:1, step:0.01}

    eta = 1. #@param{type:"number"}

    order = 4 #@param{type:"slider", min:1, max:4, step:1}

    #@markdown ${\textsf{CLIP Options}}$
    stop_at_last_layer = -6 #@param{type:"number"}
    width = 5120 #@param{type:"number"}
    height = 5120 #@param{type:"number"}
    crop_w = 0 #@param{type:"number"}
    crop_h = 0 #@param{type:"number"}
    target_width = 5120 #@param{type:"number"}
    target_height = 5120 #@param{type:"number"}

    #@markdown ${\textsf{Noise Options}}$
    denoise = 1.0 #@param{type:"number"}
    force_full_denoise = False #@param{type:"boolean"}
    disable_noise = False #@param{type:"boolean"}
    force_full_denoise = force_full_denoise if use_refiner else True
    noisedevice = "cuda:0" #@param{type:"string"}["cuda:0", "cpu"]

    #@markdown ${\textsf{REFINER OPTIONS}}$
    refinerckpt_name='sd_xl_refiner_1.0_0.9vae.safetensors' #@param{type:"string"} ["sd_xl_refiner_1.0_0.9vae.safetensors", "sd_xl_refiner_1.0.safetensors"]
    refinerforce_full_denoise = True #@param{type:"boolean"}
    refinerdisable_noise = True #@param{type:"boolean"}
    refinerwidth = 5120 #@param{type:"number"}
    refinerheight = 5120 #@param{type:"number"}
    ascore = 3.0 #@param{type:"number"}

    #@markdown ${\textsf{Controlnet Canny OPTIONS}}$
    is_controlnet = False #@param{type:"boolean"}
    controlnet_name = "sdxl_1-0_depth.safetensors" #@param{type:"string"} ["sdxl_1-0_depth.safetensors", "sdxl_1-0_canny.safetensors"]
    controlnet_image = "/content/upscaled_onyx.png" #@param{type:"string"}
    controlnet_low_threshold = 0.19 # @param {type:"slider", min:0.01, max:0.99, step:0.01}
    controlnet_high_threshold = 0.87 # @param {type:"slider", min:0.01, max:0.99, step:0.01}
    controlnet_strength = 0.59 # @param {type:"slider", min:0, max:10, step:0.01}
    controlnet_start_percent = 0 # @param {type:"slider", min:0, max:1, step:0.001}
    controlnet_end_percent = 1.0 # @param {type:"slider", min:0, max:1, step:0.001}

    #@markdown ${\textsf{Save Options}}$
    output_folder = "civit" #@param{type:"string"}
    saveprefix = f'{sampler_name}_{scheduler}'
    save_base_image = True #@param{type:"boolean"}
    save_base_image = save_base_image if use_refiner else True

    use_preview = True #@param{type:"boolean"}
    create_video_preview = True #@param{type:"boolean"}
    disable_metadata = False #@param{type:"boolean"}

    return locals()

sdxl_args = SDXL_ARGS()
sdxl_args = SimpleNamespace(**sdxl_args)

load_settings_metadata = False #@param{type:"boolean"}
if load_settings_metadata:
    import json
    from PIL import Image as pilimage
    load_settings_from_image = '/content/ComfyUI-Notebook/civit/euler_ancestral_exponential_00002_.png' #@param{type:"string"}
    png_image = pilimage.open(load_settings_from_image)
    pnginfo = png_image.info

    for key in sdxl_args.__dict__.keys():
        if key in pnginfo:
            value = json.loads(pnginfo[key])
            print(f'Adding value to key: {value}')
            setattr(sdxl_args, key, value)

with torch.inference_mode():
    try:
        out
        if sdxl_args.ckpt_name != checkpoint:
            del out
            del model
            del samples
            gc.collect()
            torch.cuda.empty_cache()
            out = loadsdxl(sdxl_args)
            checkpoint = sdxl_args.ckpt_name
        if sdxl_args.use_refiner:
            refined_out
        if sdxl_args.is_controlnet:
            control_net
    except:
        out = loadsdxl(sdxl_args)
        checkpoint = sdxl_args.ckpt_name #create checkpoint to trigger loading a new checkpoint
        control_net = None
        if sdxl_args.use_refiner:
            refined_out = loadsdxlrefiner(sdxl_args)
        if sdxl_args.is_controlnet:
            controlnet_loader = nodes.ControlNetLoader()
            control_net = controlnet_loader.load_controlnet(sdxl_args.controlnet_name)

    print('Running with settings:')
    print(f'Prompt: {sdxl_args.prompt}')
    print(f'Negative Prompt: {sdxl_args.negativeprompt}')
    for key, value in sdxl_args.__dict__.items():
        if not "prompt" in key and not "negativeprompt" in key:
            print(f'{key}: {value}')

    model, samples = runsdxl(sdxl_args, out, control_net)
    model_management.unload_model(model)
    model_management.unload_model(out[1].patcher)
    sdxlrun.get_device_memory()
    if sdxl_args.use_refiner:
        refinermodel, refinedsamples = runsdxlrefiner(sdxl_args, samples, model, refined_out)
        model_management.unload_model(refinermodel)
        model_management.unload_model(refined_out[1].patcher)
        sdxlrun.get_device_memory()

torch.cuda.empty_cache()
gc.collect()
checkpoint = sdxl_args.ckpt_name

##<left><font face="Monospace" size="6">RUN SDXL -> Single Prompt for All Schedulers and Samplers</font></left>

In [ ]:
#@title <left><font face="Monospace" size="4">Set CudaMallocAsync</font></left>
import os
env_var = "backend:cudaMallocAsync"
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = env_var
os.environ['PYTORCH_CUDA_ALLOC_CONF']

In [ ]:
#@title $ {\textsf {RUN SDXL -> Single Prompt for All Schedulers and Samplers}}$
working_dir = "/content/ComfyUI-Notebook" #@param{type:"string"}
%cd $working_dir
import gc
import torch
import random
import nodes
import sdxlrun
import ipywidgets
from types import SimpleNamespace
from comfy import model_management
from sdxlrun import runsdxl, loadsdxl
from pytorch_lightning import seed_everything
from refinersdxlrun import runsdxlrefiner, loadsdxlrefiner
from iprogress import iprogress

def SDXL_ARGS():
    prompt = "(horror art style), (gruesome), intricate concept art of a (mad scientist's laboratory:1.1) with (grotesque experiments:1.2), (flickering lights:1.3), (green colors), (repulsive), high quality, fluorescent light, stomach-churning atmosphere" #@param{type:"string"}
    negativeprompt = "close up image, (blur:1.8), fake, anime, cartoon, girl, lady, female, bad art, blurry, blurry background, bokeh, bad drawing, poor composition, poor use of textures, poor use of colors, poor art drawing, portrait, ugly, face shot, Overbaked, over contrasted, extra fingers, deformed hands, over saturated, ((3d, cartoon, anime, sketches)), (worst quality:2), (low quality:2), (normal quality:2), lowres, normal quality, ((monochrome)), ((grayscale)), bad anatomy, out of view, cut off, ugly, deformed, mutated, ((young)), EasyNegative, paintings, sketches, (worst quality:2), (low quality:2), (normal quality:2), lowres, normal quality, ((monochrome)), ((grayscale)), skin spots, acnes, skin blemishes, age spot, glans,extra fingers,fewer fingers, (ugly eyes, deformed iris, deformed pupils, fused lips and teeth:1.2), (un-detailed skin, semi-realistic, cgi, 3d, render, sketch, cartoon, drawing, anime:1.2), text, close up, cropped, out of frame, worst quality, low quality, jpeg artifacts, ugly, duplicate, morbid, mutilated, extra fingers, mutated hands, poorly drawn hands, poorly drawn face, mutation, deformed, blurry, dehydrated, bad anatomy, bad proportions, extra limbs, cloned face, disfigured, gross proportions, malformed limbs, missing arms, missing legs, extra arms, extra legs, fused fingers, too many fingers, long neck," #@param{type:"string"}
    #@markdown ${\textsf{Base Model Options}}$
    ckpt_path='/content/ComfyUI/models/checkpoints/dreamshaperXL10_alpha2XI10.safetensors'
    ckpt_name='crystalClearxl_ccxl.safetensors' #@param{type:"string"}["crystalClearxl_ccxl.safetensors", "psyanimated_xl.safetensors", "sd_xl_base_1.0_0.9vae.safetensors", "dreamshaperXL10_alpha2XI10.safetensors"]
    lora_name = "None"#@param{type:"string"}["None", "sd_xl_offset_example-lora_1.0.safetensors", "greg_rutkowski_xl_2.safetensors"]
    if lora_name == "None":
        lora_name = None
    else:
        strength_model = 0.5 #@param{type:"number"}
        strength_clip = 0.5 #@param{type:"number"}

    #@markdown ${\textsf{Base Model}}$
    steps = 35 #@param{type:"number"}
    start_step = 0 #@param{type:"number"}
    last_step = 30 #@param{type:"number"}

    #refiner model
    use_refiner =  False #@param{type:"boolean"}
    last_step = last_step if use_refiner else steps
    refiner_steps = steps
    refiner_start_step = last_step
    refiner_last_step = steps

    #@markdown ${\textsf{VAE Settings}}$
    vae_path = "/content/ComfyUI-Notebook/models/vae/sdxl_vae.safetensors" #@param{type:"string"}
    vae_decode_method = "tiled" #@param{type:"string"}["normal", "tiled"]

    #@markdown ${\textsf{Image-Batch Settings}}$
    imagewidth = 1024 #@param{type:"number"}
    imageheight = 1344   #@param{type:"number"}
    batch_size = 1 #@param{type:"number"}

    cfg = 3.5 #@param{type:"number"}
    seed = -1 #@param{type:"number"}
    if seed == -1:
        seed = random.randint(1, 432043209)
    seed = seed_everything(seed)

    #@markdown ${\textsf{Sampler Settings}}$
    sampler_name = "dpmpp_2m_sde_gpu" #@param{type:"string"} ["euler", "euler_ancestral", "heun", "dpm_2", "dpm_2_ancestral", "lms", "dpm_fast", "dpm_adaptive", "dpmpp_2s_ancestral", "dpmpp_sde", "dpmpp_sde_gpu", "dpmpp_2m", "dpmpp_2m_sde", "dpmpp_2m_sde_gpu", "ddim", "uni_pc", "uni_pc_bh2"]
    scheduler = "exponential" #@param{type:"string"} ["normal", "karras", "exponential", "simple", "ddim_uniform"]

    #@markdown ${\textsf{Sampler Options}}$
    s_churn = 0.01 # @param {type:"slider", min:0, max:1, step:0.01}
    s_tmin = 0.01 # @param {type:"slider", min:0, max:1, step:0.01}
    s_tmax = float('inf')
    s_noise = 1. # @param {type:"slider", min:0, max:1, step:0.01}

    eta = 1. #@param{type:"number"}

    order = 4 #@param{type:"slider", min:1, max:4, step:1}

    #@markdown ${\textsf{CLIP Options}}$
    stop_at_last_layer = -6 #@param{type:"number"}
    width = 5120 #@param{type:"number"}
    height = 5120 #@param{type:"number"}
    crop_w = 0 #@param{type:"number"}
    crop_h = 0 #@param{type:"number"}
    target_width = 5120 #@param{type:"number"}
    target_height = 5120 #@param{type:"number"}

    #@markdown ${\textsf{Noise Options}}$
    denoise = 1.0 #@param{type:"number"}
    force_full_denoise = False #@param{type:"boolean"}
    disable_noise = False #@param{type:"boolean"}
    force_full_denoise = force_full_denoise if use_refiner else True
    noisedevice = "cuda:0" #@param{type:"string"}["cuda:0", "cpu"]

    #@markdown ${\textsf{REFINER OPTIONS}}$
    refinerckpt_name='sd_xl_refiner_1.0_0.9vae.safetensors' #@param{type:"string"} ["sd_xl_refiner_1.0_0.9vae.safetensors", "sd_xl_refiner_1.0.safetensors"]
    refinerforce_full_denoise = True #@param{type:"boolean"}
    refinerdisable_noise = True #@param{type:"boolean"}
    refinerwidth = 5120 #@param{type:"number"}
    refinerheight = 5120 #@param{type:"number"}
    ascore = 3.0 #@param{type:"number"}

    #@markdown ${\textsf{Controlnet Canny OPTIONS}}$
    is_controlnet = False #@param{type:"boolean"}
    controlnet_name = "sdxl_1-0_depth.safetensors" #@param{type:"string"} ["sdxl_1-0_depth.safetensors", "sdxl_1-0_canny.safetensors"]
    controlnet_image = "/content/upscaled_onyx.png" #@param{type:"string"}
    controlnet_low_threshold = 0.19 # @param {type:"slider", min:0.01, max:0.99, step:0.01}
    controlnet_high_threshold = 0.87 # @param {type:"slider", min:0.01, max:0.99, step:0.01}
    controlnet_strength = 0.59 # @param {type:"slider", min:0, max:10, step:0.01}
    controlnet_start_percent = 0 # @param {type:"slider", min:0, max:1, step:0.001}
    controlnet_end_percent = 1.0 # @param {type:"slider", min:0, max:1, step:0.001}

    #@markdown ${\textsf{Save Options}}$
    output_folder = "civit" #@param{type:"string"}
    saveprefix = f'{sampler_name}_{scheduler}'
    save_base_image = True #@param{type:"boolean"}
    save_base_image = save_base_image if use_refiner else True

    use_preview = True #@param{type:"boolean"}
    create_video_preview = True #@param{type:"boolean"}
    disable_metadata = False #@param{type:"boolean"}

    return locals()

# ["euler", "euler_ancestral", "heun", "dpm_2", "dpm_2_ancestral", "lms", "dpm_fast", "dpm_adaptive",]
sampler_names = ["euler", "euler_ancestral", "heun", "dpm_2", "dpm_2_ancestral", "lms", "dpm_fast", "dpm_adaptive", "dpmpp_2s_ancestral", "dpmpp_sde", "dpmpp_sde_gpu", "dpmpp_2m", "dpmpp_2m_sde", "dpmpp_2m_sde_gpu", "ddim", "uni_pc", "uni_pc_bh2"]
schedulers = ["normal", "karras", "exponential", "simple", "ddim_uniform"]
randomize_seed = False #@param{type:"boolean"}

for sampler_name in iprogress(sampler_names, colour="synthlite"):
    for scheduler in iprogress(schedulers, colour="cotton_candy"):
        if randomize_seed:
            seed = seed_everything(random.randint(2 ** 64))
        sdxl_args = SDXL_ARGS()
        sdxl_args = SimpleNamespace(**sdxl_args)
        sdxl_args.sampler_name = sampler_name
        sdxl_args.scheduler = scheduler
        sdxl_args.saveprefix = f'{sampler_name}_{scheduler}'
        with torch.inference_mode():
            try:
                out
                if sdxl_args.ckpt_name != checkpoint:
                    del out
                    del model
                    del samples
                    gc.collect()
                    torch.cuda.empty_cache()
                    out = loadsdxl(sdxl_args)
                    checkpoint = sdxl_args.ckpt_name
                if sdxl_args.use_refiner:
                    refined_out
                if sdxl_args.is_controlnet:
                    control_net
            except:
                out = loadsdxl(sdxl_args)
                checkpoint = sdxl_args.ckpt_name #create checkpoint to trigger loading a new checkpoint
                control_net = None
                if sdxl_args.use_refiner:
                    refined_out = loadsdxlrefiner(sdxl_args)
                if sdxl_args.is_controlnet:
                    controlnet_loader = nodes.ControlNetLoader()
                    control_net = controlnet_loader.load_controlnet(sdxl_args.controlnet_name)
            print('Running with settings:')
            print(f'Prompt: {sdxl_args.prompt}')
            print(f'Negative Prompt: {sdxl_args.negativeprompt}')
            for key, value in sdxl_args.__dict__.items():
                if not "prompt" in key and not "negativeprompt" in key:
                    print(f'{key}: {value}')
            model, samples = runsdxl(sdxl_args, out, control_net)
            model_management.unload_model(model)
            model_management.unload_model(out[1].patcher)
            sdxlrun.get_device_memory()
            if sdxl_args.use_refiner:
                refinermodel, refinedsamples = runsdxlrefiner(sdxl_args, samples, model, refined_out)
                model_management.unload_model(refinermodel)
                model_management.unload_model(refined_out[1].patcher)
                sdxlrun.get_device_memory()

        torch.cuda.empty_cache()
        gc.collect()
        sdxl_args.seed = None
        checkpoint = sdxl_args.ckpt_name

##<left><font face="Monospace" size="6">Run SDXL -> Multi-Prompt</font></left>

In [ ]:
#@title <left><font face="Monospace" size="4">Set CudaMallocAsync</font></left>
import os
env_var = "backend:cudaMallocAsync"
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = env_var
os.environ['PYTORCH_CUDA_ALLOC_CONF']

In [ ]:
prompts = {
    1: "(surrealist art style), (dreamlike), intricate concept art of a (floating island:1.1) with (waterfalls:1.1), (clouds:1.1), (pastel colors), (ethereal), high quality, soft light, enchanting atmosphere.",
    2: "(pop art style), (bold), intricate concept art of a (retro diner:1.1) with (jukebox:1.2), (checkerboard floor:1.2), (bright colors), (lively), high quality, neon light, cheerful atmosphere.",
    3: "(impressionist art style), (romantic), intricate concept art of a (couple walking:1.1) in a (rainy cityscape:1.1), (umbrella:1.2), (cool colors), (dynamic), high quality, street light, captivating atmosphere.",
    4: "(abstract art style), (thought-provoking), intricate concept art of a (geometric shapes:1.1) in a (chaotic arrangement:1.2), (vibrant colors), (energetic), high quality, harsh light, thrilling atmosphere.",
    5: "(realist art style), (nostalgic), intricate concept art of a (old train station:1.1) with (steam engine:1.2), (passengers:1.3), (earthy colors), (poised), high quality, natural light, tranquil atmosphere.",
    6: "(cubist art style), (intriguing), intricate concept art of a (still life arrangement:1.15) with (fruit bowl:1.2), (wine bottle:1.3), (neutral colors), (elegant), high quality, soft light, mesmerizing atmosphere.",
    7: "(expressionist art style), (emotional), intricate concept art of a (lonely figure:1.2) on a (cliff edge:1.3) overlooking a stormy sea, dark colors, intense, high quality, moonlight, awe-inspiring atmosphere.",
    8: "(art nouveau style), intricate concept of a(flowers :1.1) and vines entwining around a(nouveau lamp post :1.3) , pastel cityscape , pastel colors , graceful , high quality , filtered light , lively atmosphere .",
    9: "(art deco style) intricate concept of a(flapper girl :1.3) dancing in a(speakeasy :1.4) , vibrant cityscape , vibrant colors , animated , high quality , spotlight , lively atmosphere .",
    10: "(gothic art style) intricate concept of a(stained glass window :1.2) in a(cathedral :1.5) with sunlight streaming through .",
}

negative_prompts = {
    1: "bad art, blurry, blurry background, bad drawing, poor composition, poor use of textures, poor use of colors, poor art drawing, portrait, ugly, face shot, Overbaked, over contrasted, over saturated, ((3d, cartoon, anime, sketches)), (worst quality:2), (low quality:2), (normal quality:2), lowres, normal quality, ((monochrome)), ((grayscale)), bad anatomy, out of view, cut off, ugly, deformed, mutated, ((young)), EasyNegative, paintings, sketches, (worst quality:2), (low quality:2), (normal quality:2), lowres, normal quality, ((monochrome)), ((grayscale)), skin spots, acnes, skin blemishes, age spot, glans,extra fingers,fewer fingers, (ugly eyes, deformed iris, deformed pupils, fused lips and teeth:1.2), (un-detailed skin, semi-realistic, cgi, 3d, render, sketch, cartoon, drawing, anime:1.2), text, close up, cropped, out of frame, worst quality, low quality, jpeg artifacts, ugly, duplicate, morbid, mutilated, extra fingers, mutated hands, poorly drawn hands, poorly drawn face, mutation, deformed, blurry, dehydrated, bad anatomy, bad proportions, extra limbs, cloned face, disfigured, gross proportions, malformed limbs, missing arms, missing legs, extra arms, extra legs, fused fingers, too many fingers, long neck,",
    2: "bad art, blurry, blurry background, bad drawing, poor composition, poor use of textures, poor use of colors, poor art drawing, portrait, ugly, face shot, Overbaked, over contrasted, over saturated, ((3d, cartoon, anime, sketches)), (worst quality:2), (low quality:2), (normal quality:2), lowres, normal quality, ((monochrome)), ((grayscale)), bad anatomy, out of view, cut off, ugly, deformed, mutated, ((young)), EasyNegative, paintings, sketches, (worst quality:2), (low quality:2), (normal quality:2), lowres, normal quality, ((monochrome)), ((grayscale)), skin spots, acnes, skin blemishes, age spot, glans,extra fingers,fewer fingers, (ugly eyes, deformed iris, deformed pupils, fused lips and teeth:1.2), (un-detailed skin, semi-realistic, cgi, 3d, render, sketch, cartoon, drawing, anime:1.2), text, close up, cropped, out of frame, worst quality, low quality, jpeg artifacts, ugly, duplicate, morbid, mutilated, extra fingers, mutated hands, poorly drawn hands, poorly drawn face, mutation, deformed, blurry, dehydrated, bad anatomy, bad proportions, extra limbs, cloned face, disfigured, gross proportions, malformed limbs, missing arms, missing legs, extra arms, extra legs, fused fingers, too many fingers, long neck,",
    3: "bad art, blurry, blurry background, bad drawing, poor composition, poor use of textures, poor use of colors, poor art drawing, portrait, ugly, face shot, Overbaked, over contrasted, over saturated, ((3d, cartoon, anime, sketches)), (worst quality:2), (low quality:2), (normal quality:2), lowres, normal quality, ((monochrome)), ((grayscale)), bad anatomy, out of view, cut off, ugly, deformed, mutated, ((young)), EasyNegative, paintings, sketches, (worst quality:2), (low quality:2), (normal quality:2), lowres, normal quality, ((monochrome)), ((grayscale)), skin spots, acnes, skin blemishes, age spot, glans,extra fingers,fewer fingers, (ugly eyes, deformed iris, deformed pupils, fused lips and teeth:1.2), (un-detailed skin, semi-realistic, cgi, 3d, render, sketch, cartoon, drawing, anime:1.2), text, close up, cropped, out of frame, worst quality, low quality, jpeg artifacts, ugly, duplicate, morbid, mutilated, extra fingers, mutated hands, poorly drawn hands, poorly drawn face, mutation, deformed, blurry, dehydrated, bad anatomy, bad proportions, extra limbs, cloned face, disfigured, gross proportions, malformed limbs, missing arms, missing legs, extra arms, extra legs, fused fingers, too many fingers, long neck,",
    4: "bad art, blurry, blurry background, bad drawing, poor composition, poor use of textures, poor use of colors, poor art drawing, portrait, ugly, face shot, Overbaked, over contrasted, over saturated, ((3d, cartoon, anime, sketches)), (worst quality:2), (low quality:2), (normal quality:2), lowres, normal quality, ((monochrome)), ((grayscale)), bad anatomy, out of view, cut off, ugly, deformed, mutated, ((young)), EasyNegative, paintings, sketches, (worst quality:2), (low quality:2), (normal quality:2), lowres, normal quality, ((monochrome)), ((grayscale)), skin spots, acnes, skin blemishes, age spot, glans,extra fingers,fewer fingers, (ugly eyes, deformed iris, deformed pupils, fused lips and teeth:1.2), (un-detailed skin, semi-realistic, cgi, 3d, render, sketch, cartoon, drawing, anime:1.2), text, close up, cropped, out of frame, worst quality, low quality, jpeg artifacts, ugly, duplicate, morbid, mutilated, extra fingers, mutated hands, poorly drawn hands, poorly drawn face, mutation, deformed, blurry, dehydrated, bad anatomy, bad proportions, extra limbs, cloned face, disfigured, gross proportions, malformed limbs, missing arms, missing legs, extra arms, extra legs, fused fingers, too many fingers, long neck,",
    5: "bad art, blurry, blurry background, bad drawing, poor composition, poor use of textures, poor use of colors, poor art drawing, portrait, ugly, face shot, Overbaked, over contrasted, over saturated, ((3d, cartoon, anime, sketches)), (worst quality:2), (low quality:2), (normal quality:2), lowres, normal quality, ((monochrome)), ((grayscale)), bad anatomy, out of view, cut off, ugly, deformed, mutated, ((young)), EasyNegative, paintings, sketches, (worst quality:2), (low quality:2), (normal quality:2), lowres, normal quality, ((monochrome)), ((grayscale)), skin spots, acnes, skin blemishes, age spot, glans,extra fingers,fewer fingers, (ugly eyes, deformed iris, deformed pupils, fused lips and teeth:1.2), (un-detailed skin, semi-realistic, cgi, 3d, render, sketch, cartoon, drawing, anime:1.2), text, close up, cropped, out of frame, worst quality, low quality, jpeg artifacts, ugly, duplicate, morbid, mutilated, extra fingers, mutated hands, poorly drawn hands, poorly drawn face, mutation, deformed, blurry, dehydrated, bad anatomy, bad proportions, extra limbs, cloned face, disfigured, gross proportions, malformed limbs, missing arms, missing legs, extra arms, extra legs, fused fingers, too many fingers, long neck,",
    6: "bad art, blurry, blurry background, bad drawing, poor composition, poor use of textures, poor use of colors, poor art drawing, portrait, ugly, face shot, Overbaked, over contrasted, over saturated, ((3d, cartoon, anime, sketches)), (worst quality:2), (low quality:2), (normal quality:2), lowres, normal quality, ((monochrome)), ((grayscale)), bad anatomy, out of view, cut off, ugly, deformed, mutated, ((young)), EasyNegative, paintings, sketches, (worst quality:2), (low quality:2), (normal quality:2), lowres, normal quality, ((monochrome)), ((grayscale)), skin spots, acnes, skin blemishes, age spot, glans,extra fingers,fewer fingers, (ugly eyes, deformed iris, deformed pupils, fused lips and teeth:1.2), (un-detailed skin, semi-realistic, cgi, 3d, render, sketch, cartoon, drawing, anime:1.2), text, close up, cropped, out of frame, worst quality, low quality, jpeg artifacts, ugly, duplicate, morbid, mutilated, extra fingers, mutated hands, poorly drawn hands, poorly drawn face, mutation, deformed, blurry, dehydrated, bad anatomy, bad proportions, extra limbs, cloned face, disfigured, gross proportions, malformed limbs, missing arms, missing legs, extra arms, extra legs, fused fingers, too many fingers, long neck,",
    7: "bad art, blurry, blurry background, bad drawing, poor composition, poor use of textures, poor use of colors, poor art drawing, portrait, ugly, face shot, Overbaked, over contrasted, over saturated, ((3d, cartoon, anime, sketches)), (worst quality:2), (low quality:2), (normal quality:2), lowres, normal quality, ((monochrome)), ((grayscale)), bad anatomy, out of view, cut off, ugly, deformed, mutated, ((young)), EasyNegative, paintings, sketches, (worst quality:2), (low quality:2), (normal quality:2), lowres, normal quality, ((monochrome)), ((grayscale)), skin spots, acnes, skin blemishes, age spot, glans,extra fingers,fewer fingers, (ugly eyes, deformed iris, deformed pupils, fused lips and teeth:1.2), (un-detailed skin, semi-realistic, cgi, 3d, render, sketch, cartoon, drawing, anime:1.2), text, close up, cropped, out of frame, worst quality, low quality, jpeg artifacts, ugly, duplicate, morbid, mutilated, extra fingers, mutated hands, poorly drawn hands, poorly drawn face, mutation, deformed, blurry, dehydrated, bad anatomy, bad proportions, extra limbs, cloned face, disfigured, gross proportions, malformed limbs, missing arms, missing legs, extra arms, extra legs, fused fingers, too many fingers, long neck,",
    8: "bad art, blurry, blurry background, bad drawing, poor composition, poor use of textures, poor use of colors, poor art drawing, portrait, ugly, face shot, Overbaked, over contrasted, over saturated, ((3d, cartoon, anime, sketches)), (worst quality:2), (low quality:2), (normal quality:2), lowres, normal quality, ((monochrome)), ((grayscale)), bad anatomy, out of view, cut off, ugly, deformed, mutated, ((young)), EasyNegative, paintings, sketches, (worst quality:2), (low quality:2), (normal quality:2), lowres, normal quality, ((monochrome)), ((grayscale)), skin spots, acnes, skin blemishes, age spot, glans,extra fingers,fewer fingers, (ugly eyes, deformed iris, deformed pupils, fused lips and teeth:1.2), (un-detailed skin, semi-realistic, cgi, 3d, render, sketch, cartoon, drawing, anime:1.2), text, close up, cropped, out of frame, worst quality, low quality, jpeg artifacts, ugly, duplicate, morbid, mutilated, extra fingers, mutated hands, poorly drawn hands, poorly drawn face, mutation, deformed, blurry, dehydrated, bad anatomy, bad proportions, extra limbs, cloned face, disfigured, gross proportions, malformed limbs, missing arms, missing legs, extra arms, extra legs, fused fingers, too many fingers, long neck,",
    9: "bad art, blurry, blurry background, bad drawing, poor composition, poor use of textures, poor use of colors, poor art drawing, portrait, ugly, face shot, Overbaked, over contrasted, over saturated, ((3d, cartoon, anime, sketches)), (worst quality:2), (low quality:2), (normal quality:2), lowres, normal quality, ((monochrome)), ((grayscale)), bad anatomy, out of view, cut off, ugly, deformed, mutated, ((young)), EasyNegative, paintings, sketches, (worst quality:2), (low quality:2), (normal quality:2), lowres, normal quality, ((monochrome)), ((grayscale)), skin spots, acnes, skin blemishes, age spot, glans,extra fingers,fewer fingers, (ugly eyes, deformed iris, deformed pupils, fused lips and teeth:1.2), (un-detailed skin, semi-realistic, cgi, 3d, render, sketch, cartoon, drawing, anime:1.2), text, close up, cropped, out of frame, worst quality, low quality, jpeg artifacts, ugly, duplicate, morbid, mutilated, extra fingers, mutated hands, poorly drawn hands, poorly drawn face, mutation, deformed, blurry, dehydrated, bad anatomy, bad proportions, extra limbs, cloned face, disfigured, gross proportions, malformed limbs, missing arms, missing legs, extra arms, extra legs, fused fingers, too many fingers, long neck,",
    10: "bad art, blurry, blurry background, bad drawing, poor composition, poor use of textures, poor use of colors, poor art drawing, portrait, ugly, face shot, Overbaked, over contrasted, over saturated, ((3d, cartoon, anime, sketches)), (worst quality:2), (low quality:2), (normal quality:2), lowres, normal quality, ((monochrome)), ((grayscale)), bad anatomy, out of view, cut off, ugly, deformed, mutated, ((young)), EasyNegative, paintings, sketches, (worst quality:2), (low quality:2), (normal quality:2), lowres, normal quality, ((monochrome)), ((grayscale)), skin spots, acnes, skin blemishes, age spot, glans,extra fingers,fewer fingers, (ugly eyes, deformed iris, deformed pupils, fused lips and teeth:1.2), (un-detailed skin, semi-realistic, cgi, 3d, render, sketch, cartoon, drawing, anime:1.2), text, close up, cropped, out of frame, worst quality, low quality, jpeg artifacts, ugly, duplicate, morbid, mutilated, extra fingers, mutated hands, poorly drawn hands, poorly drawn face, mutation, deformed, blurry, dehydrated, bad anatomy, bad proportions, extra limbs, cloned face, disfigured, gross proportions, malformed limbs, missing arms, missing legs, extra arms, extra legs, fused fingers, too many fingers, long neck,",
}

In [ ]:
#@title <left><font face="Monospace" size="4">Run SDXL -> Multi-Prompt</font></left>
working_dir = "/content/ComfyUI-Notebook" #@param{type:"string"}
%cd $working_dir
import gc
import torch
import random
import sdxlrun
from types import SimpleNamespace
from comfy import model_management
from sdxlrun import runsdxl, loadsdxl
from pytorch_lightning import seed_everything
from refinersdxlrun import runsdxlrefiner, loadsdxlrefiner
from iprogress import iprogress

def SDXL_ARGS():
    prompt = "(horror art style), (gruesome), intricate concept art of a (mad scientist's laboratory:1.1) with (grotesque experiments:1.2), (flickering lights:1.3), (green colors), (repulsive), high quality, fluorescent light, stomach-churning atmosphere" #@param{type:"string"}
    negativeprompt = "close up image, (blur:1.8), fake, anime, cartoon, girl, lady, female, bad art, blurry, blurry background, bokeh, bad drawing, poor composition, poor use of textures, poor use of colors, poor art drawing, portrait, ugly, face shot, Overbaked, over contrasted, extra fingers, deformed hands, over saturated, ((3d, cartoon, anime, sketches)), (worst quality:2), (low quality:2), (normal quality:2), lowres, normal quality, ((monochrome)), ((grayscale)), bad anatomy, out of view, cut off, ugly, deformed, mutated, ((young)), EasyNegative, paintings, sketches, (worst quality:2), (low quality:2), (normal quality:2), lowres, normal quality, ((monochrome)), ((grayscale)), skin spots, acnes, skin blemishes, age spot, glans,extra fingers,fewer fingers, (ugly eyes, deformed iris, deformed pupils, fused lips and teeth:1.2), (un-detailed skin, semi-realistic, cgi, 3d, render, sketch, cartoon, drawing, anime:1.2), text, close up, cropped, out of frame, worst quality, low quality, jpeg artifacts, ugly, duplicate, morbid, mutilated, extra fingers, mutated hands, poorly drawn hands, poorly drawn face, mutation, deformed, blurry, dehydrated, bad anatomy, bad proportions, extra limbs, cloned face, disfigured, gross proportions, malformed limbs, missing arms, missing legs, extra arms, extra legs, fused fingers, too many fingers, long neck," #@param{type:"string"}
    #@markdown ${\textsf{Base Model Options}}$
    ckpt_path='/content/ComfyUI/models/checkpoints/dreamshaperXL10_alpha2XI10.safetensors'
    ckpt_name='crystalClearxl_ccxl.safetensors' #@param{type:"string"}["crystalClearxl_ccxl.safetensors", "psyanimated_xl.safetensors", "sd_xl_base_1.0_0.9vae.safetensors", "dreamshaperXL10_alpha2XI10.safetensors"]
    lora_name = "None"#@param{type:"string"}["None", "sd_xl_offset_example-lora_1.0.safetensors", "greg_rutkowski_xl_2.safetensors"]
    if lora_name == "None":
        lora_name = None
    else:
        strength_model = 0.5 #@param{type:"number"}
        strength_clip = 0.5 #@param{type:"number"}

    #@markdown ${\textsf{Base Model}}$
    steps = 35 #@param{type:"number"}
    start_step = 0 #@param{type:"number"}
    last_step = 30 #@param{type:"number"}

    #refiner model
    use_refiner =  False #@param{type:"boolean"}
    last_step = last_step if use_refiner else steps
    refiner_steps = steps
    refiner_start_step = last_step
    refiner_last_step = steps

    #@markdown ${\textsf{VAE Settings}}$
    vae_path = "/content/ComfyUI-Notebook/models/vae/sdxl_vae.safetensors" #@param{type:"string"}
    vae_decode_method = "tiled" #@param{type:"string"}["normal", "tiled"]

    #@markdown ${\textsf{Image-Batch Settings}}$
    imagewidth = 1024 #@param{type:"number"}
    imageheight = 1344   #@param{type:"number"}
    batch_size = 1 #@param{type:"number"}

    cfg = 3.5 #@param{type:"number"}
    seed = -1 #@param{type:"number"}
    if seed == -1:
        seed = random.randint(1, 432043209)
    seed = seed_everything(seed)

    #@markdown ${\textsf{Sampler Settings}}$
    sampler_name = "dpmpp_2m_sde_gpu" #@param{type:"string"} ["euler", "euler_ancestral", "heun", "dpm_2", "dpm_2_ancestral", "lms", "dpm_fast", "dpm_adaptive", "dpmpp_2s_ancestral", "dpmpp_sde", "dpmpp_sde_gpu", "dpmpp_2m", "dpmpp_2m_sde", "dpmpp_2m_sde_gpu", "ddim", "uni_pc", "uni_pc_bh2"]
    scheduler = "exponential" #@param{type:"string"} ["normal", "karras", "exponential", "simple", "ddim_uniform"]

    #@markdown ${\textsf{Sampler Options}}$
    s_churn = 0.01 # @param {type:"slider", min:0, max:1, step:0.01}
    s_tmin = 0.01 # @param {type:"slider", min:0, max:1, step:0.01}
    s_tmax = float('inf')
    s_noise = 1. # @param {type:"slider", min:0, max:1, step:0.01}

    eta = 1. #@param{type:"number"}

    order = 4 #@param{type:"slider", min:1, max:4, step:1}

    #@markdown ${\textsf{CLIP Options}}$
    stop_at_last_layer = -6 #@param{type:"number"}
    width = 5120 #@param{type:"number"}
    height = 5120 #@param{type:"number"}
    crop_w = 0 #@param{type:"number"}
    crop_h = 0 #@param{type:"number"}
    target_width = 5120 #@param{type:"number"}
    target_height = 5120 #@param{type:"number"}

    #@markdown ${\textsf{Noise Options}}$
    denoise = 1.0 #@param{type:"number"}
    force_full_denoise = False #@param{type:"boolean"}
    disable_noise = False #@param{type:"boolean"}
    force_full_denoise = force_full_denoise if use_refiner else True
    noisedevice = "cuda:0" #@param{type:"string"}["cuda:0", "cpu"]

    #@markdown ${\textsf{REFINER OPTIONS}}$
    refinerckpt_name='sd_xl_refiner_1.0_0.9vae.safetensors' #@param{type:"string"} ["sd_xl_refiner_1.0_0.9vae.safetensors", "sd_xl_refiner_1.0.safetensors"]
    refinerforce_full_denoise = True #@param{type:"boolean"}
    refinerdisable_noise = True #@param{type:"boolean"}
    refinerwidth = 5120 #@param{type:"number"}
    refinerheight = 5120 #@param{type:"number"}
    ascore = 3.0 #@param{type:"number"}

    #@markdown ${\textsf{Controlnet Canny OPTIONS}}$
    is_controlnet = False #@param{type:"boolean"}
    controlnet_name = "sdxl_1-0_depth.safetensors" #@param{type:"string"} ["sdxl_1-0_depth.safetensors", "sdxl_1-0_canny.safetensors"]
    controlnet_image = "/content/upscaled_onyx.png" #@param{type:"string"}
    controlnet_low_threshold = 0.19 # @param {type:"slider", min:0.01, max:0.99, step:0.01}
    controlnet_high_threshold = 0.87 # @param {type:"slider", min:0.01, max:0.99, step:0.01}
    controlnet_strength = 0.59 # @param {type:"slider", min:0, max:10, step:0.01}
    controlnet_start_percent = 0 # @param {type:"slider", min:0, max:1, step:0.001}
    controlnet_end_percent = 1.0 # @param {type:"slider", min:0, max:1, step:0.001}

    #@markdown ${\textsf{Save Options}}$
    output_folder = "civit" #@param{type:"string"}
    saveprefix = f'{sampler_name}_{scheduler}'
    save_base_image = True #@param{type:"boolean"}
    save_base_image = save_base_image if use_refiner else True

    use_preview = True #@param{type:"boolean"}
    create_video_preview = True #@param{type:"boolean"}
    disable_metadata = False #@param{type:"boolean"}

    return locals()

sdxl_args = SDXL_ARGS()
sdxl_args = SimpleNamespace(**sdxl_args)
randomize_seed = True #@param{type:"boolean"}

key_count=1
for prompt_id, prompt in prompts.items():
    sdxl_args.prompt = prompt
    sdxl_args.negativeprompt = negative_prompts[key_count]
    print(f'Running with Prompt: {sdxl_args.prompt}')
    print(f'Running with Negative Prompt: {sdxl_args.negativeprompt}')
    if randomize_seed:
        sdxl_args.seed = seed_everything(random.randint(1, 2 ** 64))

    with torch.inference_mode():
        try:
            out
            if sdxl_args.ckpt_name != checkpoint:
                del out
                del model
                del samples
                gc.collect()
                torch.cuda.empty_cache()
                out = loadsdxl(sdxl_args)
                checkpoint = sdxl_args.ckpt_name
            if sdxl_args.use_refiner:
                refined_out
            if sdxl_args.is_controlnet:
                control_net
        except:
            out = loadsdxl(sdxl_args)
            checkpoint = sdxl_args.ckpt_name #create checkpoint to trigger loading a new checkpoint
            control_net = None
            if sdxl_args.use_refiner:
                refined_out = loadsdxlrefiner(sdxl_args)
            if sdxl_args.is_controlnet:
                controlnet_loader = nodes.ControlNetLoader()
                control_net = controlnet_loader.load_controlnet(sdxl_args.controlnet_name)

        print('Running with settings:')
        print(f'Prompt: {sdxl_args.prompt}')
        print(f'Negative Prompt: {sdxl_args.negativeprompt}')
        for key, value in sdxl_args.__dict__.items():
            if not "prompt" in key and not "negativeprompt" in key:
                print(f'{key}: {value}')

        model, samples = runsdxl(sdxl_args, out, control_net)
        model_management.unload_model(model)
        model_management.unload_model(out[1].patcher)
        sdxlrun.get_device_memory()
        if sdxl_args.use_refiner:
            refinermodel, refinedsamples = runsdxlrefiner(sdxl_args, samples, model, refined_out)
            model_management.unload_model(refinermodel)
            model_management.unload_model(refined_out[1].patcher)
            sdxlrun.get_device_memory()

        torch.cuda.empty_cache()
        gc.collect()
        sdxl_args.seed = None
        checkpoint = sdxl_args.ckpt_name

#<left><font face="Monospace" size="16"><b>Model Merging</b></font></left>

In [ ]:
#@title <left><font face="Monospace" size="4">Set CudaMallocAsync</font></left>
import os
env_var = "backend:cudaMallocAsync"
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = env_var

In [ ]:
working_dir = '/content/ComfyUI-Notebook'
%cd $working_dir
import os
import gc
import sys
import torch
from sdxlrun import loadsdxl
from types import SimpleNamespace
from comfy_extras.nodes_model_merging import ModelMergeSimple, CheckpointSave

filename_prefix = 'crystal_dreamshaper'
ratio = 0.5
prompt = None
extra_pnginfo = None

def SDXL_ARGS():
    ckpt_name = 'crystalClearxl_ccxl.safetensors'
    lora_name = None
    return locals()

sdxl_args = SDXL_ARGS()
sdxl_args = SimpleNamespace(**sdxl_args)

model_merger = ModelMergeSimple()
checkpoint_saver = CheckpointSave()

out = loadsdxl(sdxl_args)
_, clip1, vae1, clipvision = out
sdxl_args.ckpt_name = 'dreamshaperXL10_alpha2XI10.safetensors'
out_1 = loadsdxl(sdxl_args)

output = model_merger.merge(out[0], out_1[0], ratio)

del out
del out_1

gc.collect()
torch.cuda.empty_cache()
checkpoint_saver.save(output[0], clip1, vae1, filename_prefix, prompt=prompt, extra_pnginfo=extra_pnginfo)

#<left><font face="Monospace" size="16"><b>ESRGAN</b></font></left>

✅(Use Multiple Models with Trained ESRGAN Models)✅

This Implementation offers the ability to use different

ESRGAN models to achieve the upscaling result you may desire

and otherwise not get using a single upscaler.

In [ ]:
#@title #<left><font face="Monospace" size="4">Install Dependencies and Requirements</font></left>
import os
import sys
import time
import subprocess
from rich.progress import track

if os.getcwd().startswith("/content"):
    HOME = '/content/'
else:
    HOME = '/workspace/'

def installesrgan():
    packages = [
        'opencv_python numpy torch rich typer'
    ]
    for package in track(packages, total=len(packages), description='Installing'):
        print(f"\033[38;2;255;192;203m..installing {package}")
        subprocess.check_call([sys.executable, '-m', 'pip', 'install'] + package.split())

install_esrgan = True #@param{type:"boolean"}
if install_esrgan:
    subprocess.run(['git', 'clone', 'https://github.com/AIManifest/ESRGAN.git'])
    installesrgan()

ESRGAN_PATH = 'ESRGAN/'
ESRGAN_DIR = os.path.join(HOME, ESRGAN_PATH)
os.chdir(ESRGAN_DIR)
sys.path.extend(ESRGAN_PATH)

In [ ]:
#@title <left><font face="Monospace" size="4">Run ESRGAN Upscaling</font></left>
import os
import sys
from pathlib import Path
from types import SimpleNamespace
from upscale import AlphaOptions, SeamlessOptions
from upscale_manifest import esrgan_upscale
from PIL import Image
from tqdm.auto import tqdm
from io import BytesIO

def get_ext(image_string):
    exts = [".png", ".jpg", ".jpeg", ".tiff"]
    image_ext = os.path.splitext(image_string)[1]
    if image_ext in exts:
        return True
    else:
        return False


use_upscale_interpolation = True #@param {type:"boolean"}
esrgan_input = "/content/jakodar/jakodar_Normal_BAE_00002.png" #@param {type:"string"}
if not os.path.exists(esrgan_input):
  os.makedirs(esrgan_input)
esrgan_output = "/content/" #@param {type:"string"}
if not os.path.exists(esrgan_output):
  os.makedirs(esrgan_output)

esrgan_model_1 = "4x-UltraSharp.pth" #@param ['4x-UltraSharp.pth', '4x-UltraMix_Balanced.pth', 'RRDB_ESRGAN_x4.pth', '4x_FatalPixels_340000_G.pth', '1x_DeJpeg_Fatality_PlusULTRA_200000_G.pth', '4x_fatal_Anime_500000_G.pth', '4x-UltraSharp.pth', '4x_NMKD-YandereNeo-Lite_320k.pth', 'RRDB_PSNR_x4.pth']
esrgan_model_1_visibility = 70 #@param {type:"slider", min:5, max:100, step:5}
esrgan_model_2 = "4x-UltraMix_Balanced.pth" #@param ['4x_Ultrasharp.pth', '4x-UltraMix_Balanced.pth', 'RRDB_ESRGAN_x4.pth', '4x_FatalPixels_340000_G.pth', '1x_DeJpeg_Fatality_PlusULTRA_200000_G.pth', '4x_fatal_Anime_500000_G.pth', '4x-UltraSharp.pth', '4x_NMKD-YandereNeo-Lite_320k.pth', 'RRDB_PSNR_x4.pth']
esrgan_model_2_visibility = 30 #@param {type:"slider", min:5, max:100, step:5}
esrgan_upscale_factor = 4 #@param{type:"number"}
esrgan_alpha_threshold = 0.5 #@param{type:"slider", min:0.01, max:2.00}
esrgan_alpha_boundary_offset = 0.2 #@param{type:"slider", min:0.01, max:2.00}
esrgan_device_id = 0 #@param{type:"number"}[0, 1, 2, 3, 4, 5, 6]
esrgan_reverse_input = False #@param{type:"boolean"}
esrgan_skip_existing_files = False #@param{type:"boolean"}
esrgan_delete_input = False #@param{type:"boolean"}
esrgan_seamless_mode = SeamlessOptions.TILE #@param[SeamlessOptions.TILE, SeamlessOptions.REPLICATE, SeamlessOptions.ALPHA_PAD]
esrgan_fp16_mode = False #@param{type:"boolean"}
esrgan_cpu_mode = False #@param{type:"boolean"}
esrgan_cache_max_split_depth = False #@param{type:"boolean"}
esrgan_alpha_options = AlphaOptions.BG_DIFFERENCE #@param[AlphaOptions.BG_DIFFERENCE, AlphaOptions.ALPHA_SEPARATELY, AlphaOptions.SWAPPING, None]
esrgan_binary_alpha = False #@param{type:"boolean"}
esrgan_ternary_alpha = False #@param{type:"boolean"}
esrgan_verbose_mode = True #@param{type:"boolean"}

if use_upscale_interpolation:
    esrgan_model_1 = f'{esrgan_model_1}@{esrgan_model_1_visibility}' + "|" + f'{esrgan_model_2}@{esrgan_model_2_visibility}'

if esrgan_model_1_visibility + esrgan_model_2_visibility != 100:
    print("\033[0;31mERROR: The sum of model visibility percentages must be 100.\033[0m")

def esrgan_args():
    model = esrgan_model_1
    input = Path(esrgan_input)
    output = Path(esrgan_output)
    outscale = esrgan_upscale_factor
    reverse = esrgan_reverse_input
    skip_existing = esrgan_skip_existing_files
    delete_input = esrgan_delete_input
    seamless = esrgan_seamless_mode
    cpu = esrgan_cpu_mode
    fp16 = esrgan_fp16_mode
    device_id = esrgan_device_id
    cache_max_split_depth = esrgan_cache_max_split_depth
    binary_alpha = esrgan_binary_alpha
    ternary_alpha = esrgan_ternary_alpha
    alpha_threshold = esrgan_alpha_threshold
    alpha_boundary_offset = esrgan_alpha_boundary_offset
    alpha_mode = esrgan_alpha_options
    verbose = esrgan_verbose_mode
    return locals()

esrganargs = esrgan_args()
esrganargs = SimpleNamespace(**esrganargs)

esrgan_upscale(esrganargs)

display_output_images = False #@param{type:"boolean"}
if display_output_images:
    import ipywidgets
    images_dir = os.listdir(esrganargs.output)
    png_images = [upscaled_image for upscaled_image in images_dir if get_ext(upscaled_image)]
    image_widgets = []
    for upscaled_image in tqdm(png_images, total=len(png_images), desc="Displaying Upscaled Images"):
        display_image = Image.open(os.path.join(esrganargs.output, upscaled_image))
        bytes_image = BytesIO()
        display_image.save(bytes_image, format='PNG')
        image_widget = ipywidgets.Image(value=bytes_image.getvalue(), format='png')
        image_widgets.append(image_widget)
    vbox = ipywidgets.VBox(image_widgets, layout=ipywidgets.Layout(width="512px"))
    display(vbox)


#<left><font face="Monospace" size="16"><b>📹Google-Research Film Interpolation📹</b></font></left>

In [ ]:
#@title **Run this cell to initiate Drive Mount and FILM installation**
import os
import sys
import time
import subprocess
import pandas as pd
from natsort import natsorted
from tqdm.auto import tqdm
from google.colab import drive
from IPython.display import display

try:
    from loguru import logger
except:
    subprocess.run(['pip', 'install', 'loguru'])
    from loguru import logger

def gpu_info():
    outv = subprocess.run([
        'nvidia-smi',
            # these lines concatenate into a single query string
            '--query-gpu='
            'timestamp,'
            'name,'
            'utilization.gpu,'
            'utilization.memory,'
            'memory.used,'
            'memory.free,'
            ,
        '--format=csv'
        ],
        stdout=subprocess.PIPE).stdout.decode('utf-8')

    header, rec = outv.split('\n')[:-1]
    return pd.DataFrame({' '.join(k.strip().split('.')).capitalize():v for k,v in zip(header.split(','), rec.split(','))}, index=[0]).T

display(gpu_info())

def setup_FILM(uninstall=False):
    gdrive = '/content/drive'
    basedir = '/content/frame-interpolation'

    drive.mount(gdrive)

    cmd = ['git', 'clone', 'https://github.com/AIManifest/frame-interpolation.git']

    gclone = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    output, error = gclone.communicate()
    logger.info(f'{error}')

    pkgs = ['tensorflow==2.8.0', # The latest should include tensorflow-gpu
            'tensorflow-datasets==4.4.0',
            'tensorflow-addons==0.16.1',
            'absl-py==0.12.0',
            'gin-config==0.5.0',
            'parameterized==0.8.1',
            'mediapy==1.0.3',
            'scikit-image==0.19.1',
            'apache-beam',
            'google-cloud-bigquery-storage==1.1.0', # Suppresses a harmless error from beam
            'natsort==8.1.0',
            'gdown==4.5.4',
            'tqdm',
            'loguru']
    for pkg in tqdm(pkgs, total=len(pkgs), colour='pink', desc='Installing FILM: '):
        if not uninstall:
            logger.info(f"..Installing {pkg}..")
            filminstall = subprocess.Popen(['pip', 'install', pkg], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
            output, error = filminstall.communicate()
            logger.info(f'{error}')
        else:
            logger.info(f"..Uninstalling {pkg}..")
            filminstall = subprocess.Popen(['pip', 'uninstall', '-y', pkg], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
            output, error = filminstall.communicate()
            logger.info(f'{output}')
    if os.path.exists(basedir):
        os.chdir(basedir)

    subprocess.run(['pip', 'install', 'condacolab'])

FILM_setup = True #@param{type:"boolean"}

if FILM_setup:
    logger.info('..Running Setup..')
    setup_FILM(uninstall=False)
else:
    logger.info('..Uninstalling..')
    setup_FILM(uninstall=True)

In [ ]:
#@title **Run this Cell to Move Images to FILM Folder as FILM operates rooted in its own source folder**
import os
import cv2
import shutil
from loguru import logger
from tqdm.auto import tqdm
from natsort import natsorted
from IPython.display import display

def extractVideoFrames(video, framepath):
    VIDEO = video
    VIDFRAMEPATH = framepath

    if not os.path.exists(VIDFRAMEPATH):
        os.makedirs(VIDFRAMEPATH)
    # Open the video file
    cap = cv2.VideoCapture(VIDEO)

    totalvidframes = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    # Set the frame number to start from
    frame_num = 0
    pbar = tqdm(total=totalvidframes)
    while True:
        # Read a frame from the video
        ret, frame = cap.read()

        # Break the loop if we have reached the end of the video
        if not ret:
            break

        # Construct the filename for the current frame
        filename = os.path.join(VIDFRAMEPATH, f'frame{frame_num:09d}.png')

        # Save the current frame as an image file
        cv2.imwrite(filename, frame)

        # Increment the frame number
        frame_num += 1

        pbar.update(1)
    # Release the video capture object
    cap.release()
    pbar.close()

#@markdown **`input_folder` can either be a directory or a video file!**
input_folder = "/content/ComfyUI-Notebook/civit/preview_folder" #@param{type: "string"}
FILM_interpolation_folder = "/content/frame-interpolation/frames/" #@param{type: "string"}

video_extensions = ['.mp4', '.avi', '.mov', '.flv', '.wmv', '.mkv', '.webm']

if not os.path.exists(FILM_interpolation_folder):
    os.makedirs(FILM_interpolation_folder)

if os.path.splitext(input_folder)[1] in video_extensions:
    logger.info(f"..Video detected, extracting video frames to {FILM_interpolation_folder}..")
    extractVideoFrames(input_folder, FILM_interpolation_folder)

else:
    logger.info(f'Copying frames from {input_folder} and moving to {FILM_interpolation_folder}..')
    for file in tqdm(natsorted(os.listdir(input_folder))):
        if file.endswith(".png"):
            if not "depth" in file:
                new_file = os.path.join(input_folder, file)
                shutil.copy(new_file, FILM_interpolation_folder)

    xdir = natsorted(os.listdir(FILM_interpolation_folder))
    strtidx = 0

    for file in tqdm(xdir):
        src = os.path.join(FILM_interpolation_folder, file)
        dst = os.path.join(FILM_interpolation_folder, f"frame{strtidx:09d}.png")
        os.rename(src, dst)
        strtidx += 1

In [ ]:
#@title **(Optional/Recommended) Run This Cell to Split Folders to divide the images in the folders into different folders to reduce memory consumption when running FILM**

import os
import math
from loguru import logger
from tqdm.auto import tqdm
from shutil import copyfile
from natsort import natsorted

framedir = '/content/frame-interpolation'

def divide_images_into_folders(src_folder, dest_folder, num_folders):
    # Get the list of image files in the source folder
    image_files = [f for f in natsorted(os.listdir(src_folder)) if f.endswith('.png')]
    num_images = len(image_files)

    # Calculate the number of images per folder
    images_per_folder = math.ceil(num_images / num_folders)

    # Create the destination folders
    for i in range(num_folders):
        os.makedirs(os.path.join(dest_folder, f'frames{i+1}'), exist_ok=True)
        logger.info(f"Creating Folder: {os.path.join(dest_folder, f'frames{i+1}')}")

    # Divide the images into the destination folders
    for i, image_file in tqdm(enumerate(image_files), total=len(image_files)):
        folder_idx = i // images_per_folder
        src_path = os.path.join(src_folder, image_file)
        dest_path = os.path.join(dest_folder, f'frames{folder_idx+1}', image_file)
        copyfile(src_path, dest_path)
    os.rename(src_folder, "Folder")

# Example usage:
src_folder = '/content/frame-interpolation/frames' #@param{type:"string"}
dest_folder = framedir
num_folders = 5 #@param{type:"number"}
divide_images_into_folders(src_folder, dest_folder, num_folders)

In [ ]:
#@title **Run this Cell if you only want to process a certain amount of files and Remove Files Above a Given Threshold in a given directory**
import os
from natsort import natsorted

remove_files_above_threshold = False #@param{type:"boolean"}

if remove_files_above_threshold:
    directory = '/content/frame-interpolation/frames' #@param{type:"string"}
    idx_threshold = 100 #@param{type:"number"}

    for filename in natsorted(os.listdir(directory)):
        try:
            idx = int(filename.split('.')[0].split('frame')[-1]) # extract the index from the filename
            if idx > idx_threshold:
                file_path = os.path.join(directory, filename)
                os.remove(file_path)
                print(f"Deleted {file_path}")
        except ValueError:
            # if the filename does not contain an index, skip it
            continue


In [ ]:
#@title 📹 $ \color{gold} {\textsf{RUN THE FILM APPLICATION}}$ 📹
HOME = '/content/'
import os
import sys
import shutil
import importlib
import subprocess
from loguru import logger
from types import SimpleNamespace
from natsort import natsorted
import gdown

sys.path.extend(['frame-interpolation/'])

import film_interpolator_helper
from film_interpolator_helper import film_interpolator
from google.colab import runtime

importlib.reload(film_interpolator_helper)

logger.info("Beginning interpolation...")

#enter into FILM directory as it operates rooted in its own directory
HOME = '/content' if os.path.exists('/content') else os.getcwd()
basedir = "/content/frame-interpolation"

os.chdir(basedir)

print(f'Home Directory: {HOME}')

initiate_FILM_setup = True #@param{type: "boolean"}
#@markdown * Checkmark this box when using for the first time. You may need to restart runtime and rerun
FILM_folder = "/content/frame-interpolation/frames" #@param{type: "string"}
path_to_FILM_model = "/content/frame-interpolation/pretrained_models/film_net/Style/saved_model" #@param{type: "string"}
times_to_interpolate = 2 #@param {type:"slider", min:1, max:8, step:1}
FILM_fps = 60 #@param{type: "number"}
input_ext_for_FILM = "png" #@param["png", "jpg", "jpeg", "PNG", "JPG", "JPEG"]
create_output_video = True #@param{type: "boolean"}
#@markdown **Leave these empty quotes for default values. If you want to specify an amount, enter a number without quotes**
FILM_align_amount = "" #@param
FILM_block_width = "" #@param
FILM_block_height = "" #@param
if FILM_align_amount == "" or None:
    FILM_align_amount = 64
if FILM_block_width == "" or None:
    FILM_block_width = 1
if FILM_block_height == "" or None:
    FILM_block_height = 1

if initiate_FILM_setup:
    url = 'https://drive.google.com/drive/folders/1q8110-qp225asX3DQvZnfLfJPkCHmDpy?usp=sharing'
    output = 'pretrained_models'
    gdown.download_folder(url=url, output=output)

def film_inference():
    _PATTERN = FILM_folder
    _MODEL_PATH = path_to_FILM_model
    _TIMES_TO_INTERPOLATE = times_to_interpolate
    _FPS = FILM_fps
    _ALIGN = FILM_align_amount
    _BLOCK_HEIGHT = FILM_block_height
    _BLOCK_WIDTH = FILM_block_width
    _OUTPUT_VIDEO = create_output_video
    _INPUT_EXT = [str(input_ext_for_FILM)]
    return locals()

args = film_inference()
args = SimpleNamespace(**args)

for dir in natsorted(os.listdir('/content/frame-interpolation')):
    if "frames" in dir:
        args._PATTERN = dir
        logger.info(f'Interpolating folder: {dir}')
        film_interpolator(args)

#@markdown **Use this if you intend to run any other part of the notebook as dependencies may conflict, restart runtime and run Environment Setup Cell Again**
remove_all_FILM_dependencies = False #@param{type: "boolean"}
if remove_all_FILM_dependencies:
    logger.info('Uninstalling FILM Packages')
    setup_FILM(uninstall=remove_all_FILM_dependencies)

#revert back to top directory
os.chdir(HOME)

gdrive_folder = '/content/drive/MyDrive/frame-interpolation/outputs'
film_output = f'{basedir}/frames/interpolated.mp4'

if not os.path.exists(gdrive_folder):
    os.makedirs(gdrive_folder)

if os.path.exists(film_output):
    logger.info(f"..Moving your completed video from {film_output} to {gdrive_folder}..")
    shutil.copy2(film_output, gdrive_folder)
    logger.info('..Done..')


logger.info("Interpolation complete! Happy Creating!")

disconnect_after_film_interpolation = False #@param{type:"boolean"}

if disconnect_after_film_interpolation:
    if os.path.exists(film_output) and os.path.exists(gdrive_folder):
        runtime.unassign()

In [ ]:
#@title **Run this cell if you split your frames into separate folder to Process Videos If Frames Folder Was Split Into Smaller Folders**
#@markdown **This Cell will take all videos found in `BASE_FOLDER` and move them to `destination_folder`**
import os
import shutil
from loguru import logger
from natsort import natsorted

BASE_FOLDER = '/content/frame-interpolation' #@param{type:"string"}
basefolderlisted = os.listdir(BASE_FOLDER)
destination_folder = '/content/frame-interpolation/videos/' #@param{type:"string"}
if not os.path.exists(destination_folder):
    os.makedirs(destination_folder)

startidx = 1
for folder in natsorted(basefolderlisted):
    if "frames" in folder:
        logger.info(f'Detected Folder: {folder}')
        NEW_FOLDER = os.path.join(BASE_FOLDER, folder)
        for file in os.listdir(NEW_FOLDER):
            if file.endswith(".mp4"):
                logger.info(f'Detected Video: {file} -- Copying {file} to {destination_folder}')
                new_file = f"interpolated{startidx}.mp4"
                src_path = os.path.join(NEW_FOLDER, file)
                dest_path = os.path.join(destination_folder, new_file)
                shutil.copy2(src_path, dest_path)
                startidx += 1


In [ ]:
#@title **Run This Cell to Combine all of the videos generated by FILM into 1 Video `if you split your frames into separate folders`**

import os
from natsort import natsorted
from moviepy.editor import VideoFileClip, concatenate_videoclips

def concatenate_videos_in_folder(folder_path, output_file):
    # Get the list of video files in the folder
    video_files = [os.path.join(folder_path, f) for f in natsorted(os.listdir(folder_path)) if f.endswith('.mp4')]

    # Load the video files as moviepy clips
    clips = [VideoFileClip(f) for f in video_files]

    # Concatenate the clips into a single video
    final_clip = concatenate_videoclips(clips)

    # Write the concatenated video to the output file
    final_clip.write_videofile(output_file)

# Example usage:
folder_path = '/content/frame-interpolation/videos' #@param{type:"string"}
output_file = '/content/drive/MyDrive/frame-interpolation/outputs/flying.mp4' #@param{type:"string"}
concatenate_videos_in_folder(folder_path, output_file)

In [ ]:
#@title **Run this cell to Compile all FILM Frames to Video, if necessary**
import cv2
import os
from tqdm.auto import tqdm
from natsort import natsorted
folder_path = '/content/frame-interpolation/frames/interpolated_frames' #@param{type: "string"}
output_path = '/content/drive/MyDrive/fik22_5_done.mp4' #@param{type: "string"}
fps = 60 #@param{type: "number"}

images = [img for img in natsorted(os.listdir(folder_path)) if img.endswith(".png")]
frame = cv2.imread(os.path.join(folder_path, images[0]))
height, width, layers = frame.shape

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
video = cv2.VideoWriter(output_path, fourcc, fps, (width,height))

for image in tqdm(natsorted(images)):
    video.write(cv2.imread(os.path.join(folder_path, image)))

cv2.destroyAllWindows()
video.release()

In [ ]:
#@title **DISCONNECT RUNTIME IF `disconnect_forever` enabled**

disconnect_forever = True #@param{type:"boolean"}

from google.colab import runtime
try:
    from loguru import logger
except:
    running = subprocess.Popen(['pip', 'install', 'loguru'],
                               stdout=subprocess.PIPE,
                               stderr=subprocess.PIPE)
    output, error = running.communicate()
    print(error)
    from loguru import logger
if disconnect_forever:
    logger.info("DISCONNECTING.. GOODBYE")
    runtime.unassign()

#<left><font face="Monospace" size="16"><b>LICENSES</b></font></left>

##COMFYUI
 GNU GENERAL PUBLIC LICENSE
                       Version 3, 29 June 2007

 Copyright (C) 2007 Free Software Foundation, Inc. <https://fsf.org/>
 Everyone is permitted to copy and distribute verbatim copies
 of this license document, but changing it is not allowed.

                            Preamble

  The GNU General Public License is a free, copyleft license for
software and other kinds of works.

  The licenses for most software and other practical works are designed
to take away your freedom to share and change the works.  By contrast,
the GNU General Public License is intended to guarantee your freedom to
share and change all versions of a program--to make sure it remains free
software for all its users.  We, the Free Software Foundation, use the
GNU General Public License for most of our software; it applies also to
any other work released this way by its authors.  You can apply it to
your programs, too.

  When we speak of free software, we are referring to freedom, not
price.  Our General Public Licenses are designed to make sure that you
have the freedom to distribute copies of free software (and charge for
them if you wish), that you receive source code or can get it if you
want it, that you can change the software or use pieces of it in new
free programs, and that you know you can do these things.

  To protect your rights, we need to prevent others from denying you
these rights or asking you to surrender the rights.  Therefore, you have
certain responsibilities if you distribute copies of the software, or if
you modify it: responsibilities to respect the freedom of others.

  For example, if you distribute copies of such a program, whether
gratis or for a fee, you must pass on to the recipients the same
freedoms that you received.  You must make sure that they, too, receive
or can get the source code.  And you must show them these terms so they
know their rights.

  Developers that use the GNU GPL protect your rights with two steps:
(1) assert copyright on the software, and (2) offer you this License
giving you legal permission to copy, distribute and/or modify it.

  For the developers' and authors' protection, the GPL clearly explains
that there is no warranty for this free software.  For both users' and
authors' sake, the GPL requires that modified versions be marked as
changed, so that their problems will not be attributed erroneously to
authors of previous versions.

  Some devices are designed to deny users access to install or run
modified versions of the software inside them, although the manufacturer
can do so.  This is fundamentally incompatible with the aim of
protecting users' freedom to change the software.  The systematic
pattern of such abuse occurs in the area of products for individuals to
use, which is precisely where it is most unacceptable.  Therefore, we
have designed this version of the GPL to prohibit the practice for those
products.  If such problems arise substantially in other domains, we
stand ready to extend this provision to those domains in future versions
of the GPL, as needed to protect the freedom of users.

  Finally, every program is threatened constantly by software patents.
States should not allow patents to restrict development and use of
software on general-purpose computers, but in those that do, we wish to
avoid the special danger that patents applied to a free program could
make it effectively proprietary.  To prevent this, the GPL assures that
patents cannot be used to render the program non-free.

  The precise terms and conditions for copying, distribution and
modification follow.

                       TERMS AND CONDITIONS

  0. Definitions.

  "This License" refers to version 3 of the GNU General Public License.

  "Copyright" also means copyright-like laws that apply to other kinds of
works, such as semiconductor masks.

  "The Program" refers to any copyrightable work licensed under this
License.  Each licensee is addressed as "you".  "Licensees" and
"recipients" may be individuals or organizations.

  To "modify" a work means to copy from or adapt all or part of the work
in a fashion requiring copyright permission, other than the making of an
exact copy.  The resulting work is called a "modified version" of the
earlier work or a work "based on" the earlier work.

  A "covered work" means either the unmodified Program or a work based
on the Program.

  To "propagate" a work means to do anything with it that, without
permission, would make you directly or secondarily liable for
infringement under applicable copyright law, except executing it on a
computer or modifying a private copy.  Propagation includes copying,
distribution (with or without modification), making available to the
public, and in some countries other activities as well.

  To "convey" a work means any kind of propagation that enables other
parties to make or receive copies.  Mere interaction with a user through
a computer network, with no transfer of a copy, is not conveying.

  An interactive user interface displays "Appropriate Legal Notices"
to the extent that it includes a convenient and prominently visible
feature that (1) displays an appropriate copyright notice, and (2)
tells the user that there is no warranty for the work (except to the
extent that warranties are provided), that licensees may convey the
work under this License, and how to view a copy of this License.  If
the interface presents a list of user commands or options, such as a
menu, a prominent item in the list meets this criterion.

  1. Source Code.

  The "source code" for a work means the preferred form of the work
for making modifications to it.  "Object code" means any non-source
form of a work.

  A "Standard Interface" means an interface that either is an official
standard defined by a recognized standards body, or, in the case of
interfaces specified for a particular programming language, one that
is widely used among developers working in that language.

  The "System Libraries" of an executable work include anything, other
than the work as a whole, that (a) is included in the normal form of
packaging a Major Component, but which is not part of that Major
Component, and (b) serves only to enable use of the work with that
Major Component, or to implement a Standard Interface for which an
implementation is available to the public in source code form.  A
"Major Component", in this context, means a major essential component
(kernel, window system, and so on) of the specific operating system
(if any) on which the executable work runs, or a compiler used to
produce the work, or an object code interpreter used to run it.

  The "Corresponding Source" for a work in object code form means all
the source code needed to generate, install, and (for an executable
work) run the object code and to modify the work, including scripts to
control those activities.  However, it does not include the work's
System Libraries, or general-purpose tools or generally available free
programs which are used unmodified in performing those activities but
which are not part of the work.  For example, Corresponding Source
includes interface definition files associated with source files for
the work, and the source code for shared libraries and dynamically
linked subprograms that the work is specifically designed to require,
such as by intimate data communication or control flow between those
subprograms and other parts of the work.

  The Corresponding Source need not include anything that users
can regenerate automatically from other parts of the Corresponding
Source.

  The Corresponding Source for a work in source code form is that
same work.

  2. Basic Permissions.

  All rights granted under this License are granted for the term of
copyright on the Program, and are irrevocable provided the stated
conditions are met.  This License explicitly affirms your unlimited
permission to run the unmodified Program.  The output from running a
covered work is covered by this License only if the output, given its
content, constitutes a covered work.  This License acknowledges your
rights of fair use or other equivalent, as provided by copyright law.

  You may make, run and propagate covered works that you do not
convey, without conditions so long as your license otherwise remains
in force.  You may convey covered works to others for the sole purpose
of having them make modifications exclusively for you, or provide you
with facilities for running those works, provided that you comply with
the terms of this License in conveying all material for which you do
not control copyright.  Those thus making or running the covered works
for you must do so exclusively on your behalf, under your direction
and control, on terms that prohibit them from making any copies of
your copyrighted material outside their relationship with you.

  Conveying under any other circumstances is permitted solely under
the conditions stated below.  Sublicensing is not allowed; section 10
makes it unnecessary.

  3. Protecting Users' Legal Rights From Anti-Circumvention Law.

  No covered work shall be deemed part of an effective technological
measure under any applicable law fulfilling obligations under article
11 of the WIPO copyright treaty adopted on 20 December 1996, or
similar laws prohibiting or restricting circumvention of such
measures.

  When you convey a covered work, you waive any legal power to forbid
circumvention of technological measures to the extent such circumvention
is effected by exercising rights under this License with respect to
the covered work, and you disclaim any intention to limit operation or
modification of the work as a means of enforcing, against the work's
users, your or third parties' legal rights to forbid circumvention of
technological measures.

  4. Conveying Verbatim Copies.

  You may convey verbatim copies of the Program's source code as you
receive it, in any medium, provided that you conspicuously and
appropriately publish on each copy an appropriate copyright notice;
keep intact all notices stating that this License and any
non-permissive terms added in accord with section 7 apply to the code;
keep intact all notices of the absence of any warranty; and give all
recipients a copy of this License along with the Program.

  You may charge any price or no price for each copy that you convey,
and you may offer support or warranty protection for a fee.

  5. Conveying Modified Source Versions.

  You may convey a work based on the Program, or the modifications to
produce it from the Program, in the form of source code under the
terms of section 4, provided that you also meet all of these conditions:

    a) The work must carry prominent notices stating that you modified
    it, and giving a relevant date.

    b) The work must carry prominent notices stating that it is
    released under this License and any conditions added under section
    7.  This requirement modifies the requirement in section 4 to
    "keep intact all notices".

    c) You must license the entire work, as a whole, under this
    License to anyone who comes into possession of a copy.  This
    License will therefore apply, along with any applicable section 7
    additional terms, to the whole of the work, and all its parts,
    regardless of how they are packaged.  This License gives no
    permission to license the work in any other way, but it does not
    invalidate such permission if you have separately received it.

    d) If the work has interactive user interfaces, each must display
    Appropriate Legal Notices; however, if the Program has interactive
    interfaces that do not display Appropriate Legal Notices, your
    work need not make them do so.

  A compilation of a covered work with other separate and independent
works, which are not by their nature extensions of the covered work,
and which are not combined with it such as to form a larger program,
in or on a volume of a storage or distribution medium, is called an
"aggregate" if the compilation and its resulting copyright are not
used to limit the access or legal rights of the compilation's users
beyond what the individual works permit.  Inclusion of a covered work
in an aggregate does not cause this License to apply to the other
parts of the aggregate.

  6. Conveying Non-Source Forms.

  You may convey a covered work in object code form under the terms
of sections 4 and 5, provided that you also convey the
machine-readable Corresponding Source under the terms of this License,
in one of these ways:

    a) Convey the object code in, or embodied in, a physical product
    (including a physical distribution medium), accompanied by the
    Corresponding Source fixed on a durable physical medium
    customarily used for software interchange.

    b) Convey the object code in, or embodied in, a physical product
    (including a physical distribution medium), accompanied by a
    written offer, valid for at least three years and valid for as
    long as you offer spare parts or customer support for that product
    model, to give anyone who possesses the object code either (1) a
    copy of the Corresponding Source for all the software in the
    product that is covered by this License, on a durable physical
    medium customarily used for software interchange, for a price no
    more than your reasonable cost of physically performing this
    conveying of source, or (2) access to copy the
    Corresponding Source from a network server at no charge.

    c) Convey individual copies of the object code with a copy of the
    written offer to provide the Corresponding Source.  This
    alternative is allowed only occasionally and noncommercially, and
    only if you received the object code with such an offer, in accord
    with subsection 6b.

    d) Convey the object code by offering access from a designated
    place (gratis or for a charge), and offer equivalent access to the
    Corresponding Source in the same way through the same place at no
    further charge.  You need not require recipients to copy the
    Corresponding Source along with the object code.  If the place to
    copy the object code is a network server, the Corresponding Source
    may be on a different server (operated by you or a third party)
    that supports equivalent copying facilities, provided you maintain
    clear directions next to the object code saying where to find the
    Corresponding Source.  Regardless of what server hosts the
    Corresponding Source, you remain obligated to ensure that it is
    available for as long as needed to satisfy these requirements.

    e) Convey the object code using peer-to-peer transmission, provided
    you inform other peers where the object code and Corresponding
    Source of the work are being offered to the general public at no
    charge under subsection 6d.

  A separable portion of the object code, whose source code is excluded
from the Corresponding Source as a System Library, need not be
included in conveying the object code work.

  A "User Product" is either (1) a "consumer product", which means any
tangible personal property which is normally used for personal, family,
or household purposes, or (2) anything designed or sold for incorporation
into a dwelling.  In determining whether a product is a consumer product,
doubtful cases shall be resolved in favor of coverage.  For a particular
product received by a particular user, "normally used" refers to a
typical or common use of that class of product, regardless of the status
of the particular user or of the way in which the particular user
actually uses, or expects or is expected to use, the product.  A product
is a consumer product regardless of whether the product has substantial
commercial, industrial or non-consumer uses, unless such uses represent
the only significant mode of use of the product.

  "Installation Information" for a User Product means any methods,
procedures, authorization keys, or other information required to install
and execute modified versions of a covered work in that User Product from
a modified version of its Corresponding Source.  The information must
suffice to ensure that the continued functioning of the modified object
code is in no case prevented or interfered with solely because
modification has been made.

  If you convey an object code work under this section in, or with, or
specifically for use in, a User Product, and the conveying occurs as
part of a transaction in which the right of possession and use of the
User Product is transferred to the recipient in perpetuity or for a
fixed term (regardless of how the transaction is characterized), the
Corresponding Source conveyed under this section must be accompanied
by the Installation Information.  But this requirement does not apply
if neither you nor any third party retains the ability to install
modified object code on the User Product (for example, the work has
been installed in ROM).

  The requirement to provide Installation Information does not include a
requirement to continue to provide support service, warranty, or updates
for a work that has been modified or installed by the recipient, or for
the User Product in which it has been modified or installed.  Access to a
network may be denied when the modification itself materially and
adversely affects the operation of the network or violates the rules and
protocols for communication across the network.

  Corresponding Source conveyed, and Installation Information provided,
in accord with this section must be in a format that is publicly
documented (and with an implementation available to the public in
source code form), and must require no special password or key for
unpacking, reading or copying.

  7. Additional Terms.

  "Additional permissions" are terms that supplement the terms of this
License by making exceptions from one or more of its conditions.
Additional permissions that are applicable to the entire Program shall
be treated as though they were included in this License, to the extent
that they are valid under applicable law.  If additional permissions
apply only to part of the Program, that part may be used separately
under those permissions, but the entire Program remains governed by
this License without regard to the additional permissions.

  When you convey a copy of a covered work, you may at your option
remove any additional permissions from that copy, or from any part of
it.  (Additional permissions may be written to require their own
removal in certain cases when you modify the work.)  You may place
additional permissions on material, added by you to a covered work,
for which you have or can give appropriate copyright permission.

  Notwithstanding any other provision of this License, for material you
add to a covered work, you may (if authorized by the copyright holders of
that material) supplement the terms of this License with terms:

    a) Disclaiming warranty or limiting liability differently from the
    terms of sections 15 and 16 of this License; or

    b) Requiring preservation of specified reasonable legal notices or
    author attributions in that material or in the Appropriate Legal
    Notices displayed by works containing it; or

    c) Prohibiting misrepresentation of the origin of that material, or
    requiring that modified versions of such material be marked in
    reasonable ways as different from the original version; or

    d) Limiting the use for publicity purposes of names of licensors or
    authors of the material; or

    e) Declining to grant rights under trademark law for use of some
    trade names, trademarks, or service marks; or

    f) Requiring indemnification of licensors and authors of that
    material by anyone who conveys the material (or modified versions of
    it) with contractual assumptions of liability to the recipient, for
    any liability that these contractual assumptions directly impose on
    those licensors and authors.

  All other non-permissive additional terms are considered "further
restrictions" within the meaning of section 10.  If the Program as you
received it, or any part of it, contains a notice stating that it is
governed by this License along with a term that is a further
restriction, you may remove that term.  If a license document contains
a further restriction but permits relicensing or conveying under this
License, you may add to a covered work material governed by the terms
of that license document, provided that the further restriction does
not survive such relicensing or conveying.

  If you add terms to a covered work in accord with this section, you
must place, in the relevant source files, a statement of the
additional terms that apply to those files, or a notice indicating
where to find the applicable terms.

  Additional terms, permissive or non-permissive, may be stated in the
form of a separately written license, or stated as exceptions;
the above requirements apply either way.

  8. Termination.

  You may not propagate or modify a covered work except as expressly
provided under this License.  Any attempt otherwise to propagate or
modify it is void, and will automatically terminate your rights under
this License (including any patent licenses granted under the third
paragraph of section 11).

  However, if you cease all violation of this License, then your
license from a particular copyright holder is reinstated (a)
provisionally, unless and until the copyright holder explicitly and
finally terminates your license, and (b) permanently, if the copyright
holder fails to notify you of the violation by some reasonable means
prior to 60 days after the cessation.

  Moreover, your license from a particular copyright holder is
reinstated permanently if the copyright holder notifies you of the
violation by some reasonable means, this is the first time you have
received notice of violation of this License (for any work) from that
copyright holder, and you cure the violation prior to 30 days after
your receipt of the notice.

  Termination of your rights under this section does not terminate the
licenses of parties who have received copies or rights from you under
this License.  If your rights have been terminated and not permanently
reinstated, you do not qualify to receive new licenses for the same
material under section 10.

  9. Acceptance Not Required for Having Copies.

  You are not required to accept this License in order to receive or
run a copy of the Program.  Ancillary propagation of a covered work
occurring solely as a consequence of using peer-to-peer transmission
to receive a copy likewise does not require acceptance.  However,
nothing other than this License grants you permission to propagate or
modify any covered work.  These actions infringe copyright if you do
not accept this License.  Therefore, by modifying or propagating a
covered work, you indicate your acceptance of this License to do so.

  10. Automatic Licensing of Downstream Recipients.

  Each time you convey a covered work, the recipient automatically
receives a license from the original licensors, to run, modify and
propagate that work, subject to this License.  You are not responsible
for enforcing compliance by third parties with this License.

  An "entity transaction" is a transaction transferring control of an
organization, or substantially all assets of one, or subdividing an
organization, or merging organizations.  If propagation of a covered
work results from an entity transaction, each party to that
transaction who receives a copy of the work also receives whatever
licenses to the work the party's predecessor in interest had or could
give under the previous paragraph, plus a right to possession of the
Corresponding Source of the work from the predecessor in interest, if
the predecessor has it or can get it with reasonable efforts.

  You may not impose any further restrictions on the exercise of the
rights granted or affirmed under this License.  For example, you may
not impose a license fee, royalty, or other charge for exercise of
rights granted under this License, and you may not initiate litigation
(including a cross-claim or counterclaim in a lawsuit) alleging that
any patent claim is infringed by making, using, selling, offering for
sale, or importing the Program or any portion of it.

  11. Patents.

  A "contributor" is a copyright holder who authorizes use under this
License of the Program or a work on which the Program is based.  The
work thus licensed is called the contributor's "contributor version".

  A contributor's "essential patent claims" are all patent claims
owned or controlled by the contributor, whether already acquired or
hereafter acquired, that would be infringed by some manner, permitted
by this License, of making, using, or selling its contributor version,
but do not include claims that would be infringed only as a
consequence of further modification of the contributor version.  For
purposes of this definition, "control" includes the right to grant
patent sublicenses in a manner consistent with the requirements of
this License.

  Each contributor grants you a non-exclusive, worldwide, royalty-free
patent license under the contributor's essential patent claims, to
make, use, sell, offer for sale, import and otherwise run, modify and
propagate the contents of its contributor version.

  In the following three paragraphs, a "patent license" is any express
agreement or commitment, however denominated, not to enforce a patent
(such as an express permission to practice a patent or covenant not to
sue for patent infringement).  To "grant" such a patent license to a
party means to make such an agreement or commitment not to enforce a
patent against the party.

  If you convey a covered work, knowingly relying on a patent license,
and the Corresponding Source of the work is not available for anyone
to copy, free of charge and under the terms of this License, through a
publicly available network server or other readily accessible means,
then you must either (1) cause the Corresponding Source to be so
available, or (2) arrange to deprive yourself of the benefit of the
patent license for this particular work, or (3) arrange, in a manner
consistent with the requirements of this License, to extend the patent
license to downstream recipients.  "Knowingly relying" means you have
actual knowledge that, but for the patent license, your conveying the
covered work in a country, or your recipient's use of the covered work
in a country, would infringe one or more identifiable patents in that
country that you have reason to believe are valid.

  If, pursuant to or in connection with a single transaction or
arrangement, you convey, or propagate by procuring conveyance of, a
covered work, and grant a patent license to some of the parties
receiving the covered work authorizing them to use, propagate, modify
or convey a specific copy of the covered work, then the patent license
you grant is automatically extended to all recipients of the covered
work and works based on it.

  A patent license is "discriminatory" if it does not include within
the scope of its coverage, prohibits the exercise of, or is
conditioned on the non-exercise of one or more of the rights that are
specifically granted under this License.  You may not convey a covered
work if you are a party to an arrangement with a third party that is
in the business of distributing software, under which you make payment
to the third party based on the extent of your activity of conveying
the work, and under which the third party grants, to any of the
parties who would receive the covered work from you, a discriminatory
patent license (a) in connection with copies of the covered work
conveyed by you (or copies made from those copies), or (b) primarily
for and in connection with specific products or compilations that
contain the covered work, unless you entered into that arrangement,
or that patent license was granted, prior to 28 March 2007.

  Nothing in this License shall be construed as excluding or limiting
any implied license or other defenses to infringement that may
otherwise be available to you under applicable patent law.

  12. No Surrender of Others' Freedom.

  If conditions are imposed on you (whether by court order, agreement or
otherwise) that contradict the conditions of this License, they do not
excuse you from the conditions of this License.  If you cannot convey a
covered work so as to satisfy simultaneously your obligations under this
License and any other pertinent obligations, then as a consequence you may
not convey it at all.  For example, if you agree to terms that obligate you
to collect a royalty for further conveying from those to whom you convey
the Program, the only way you could satisfy both those terms and this
License would be to refrain entirely from conveying the Program.

  13. Use with the GNU Affero General Public License.

  Notwithstanding any other provision of this License, you have
permission to link or combine any covered work with a work licensed
under version 3 of the GNU Affero General Public License into a single
combined work, and to convey the resulting work.  The terms of this
License will continue to apply to the part which is the covered work,
but the special requirements of the GNU Affero General Public License,
section 13, concerning interaction through a network will apply to the
combination as such.

  14. Revised Versions of this License.

  The Free Software Foundation may publish revised and/or new versions of
the GNU General Public License from time to time.  Such new versions will
be similar in spirit to the present version, but may differ in detail to
address new problems or concerns.

  Each version is given a distinguishing version number.  If the
Program specifies that a certain numbered version of the GNU General
Public License "or any later version" applies to it, you have the
option of following the terms and conditions either of that numbered
version or of any later version published by the Free Software
Foundation.  If the Program does not specify a version number of the
GNU General Public License, you may choose any version ever published
by the Free Software Foundation.

  If the Program specifies that a proxy can decide which future
versions of the GNU General Public License can be used, that proxy's
public statement of acceptance of a version permanently authorizes you
to choose that version for the Program.

  Later license versions may give you additional or different
permissions.  However, no additional obligations are imposed on any
author or copyright holder as a result of your choosing to follow a
later version.

  15. Disclaimer of Warranty.

  THERE IS NO WARRANTY FOR THE PROGRAM, TO THE EXTENT PERMITTED BY
APPLICABLE LAW.  EXCEPT WHEN OTHERWISE STATED IN WRITING THE COPYRIGHT
HOLDERS AND/OR OTHER PARTIES PROVIDE THE PROGRAM "AS IS" WITHOUT WARRANTY
OF ANY KIND, EITHER EXPRESSED OR IMPLIED, INCLUDING, BUT NOT LIMITED TO,
THE IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR
PURPOSE.  THE ENTIRE RISK AS TO THE QUALITY AND PERFORMANCE OF THE PROGRAM
IS WITH YOU.  SHOULD THE PROGRAM PROVE DEFECTIVE, YOU ASSUME THE COST OF
ALL NECESSARY SERVICING, REPAIR OR CORRECTION.

  16. Limitation of Liability.

  IN NO EVENT UNLESS REQUIRED BY APPLICABLE LAW OR AGREED TO IN WRITING
WILL ANY COPYRIGHT HOLDER, OR ANY OTHER PARTY WHO MODIFIES AND/OR CONVEYS
THE PROGRAM AS PERMITTED ABOVE, BE LIABLE TO YOU FOR DAMAGES, INCLUDING ANY
GENERAL, SPECIAL, INCIDENTAL OR CONSEQUENTIAL DAMAGES ARISING OUT OF THE
USE OR INABILITY TO USE THE PROGRAM (INCLUDING BUT NOT LIMITED TO LOSS OF
DATA OR DATA BEING RENDERED INACCURATE OR LOSSES SUSTAINED BY YOU OR THIRD
PARTIES OR A FAILURE OF THE PROGRAM TO OPERATE WITH ANY OTHER PROGRAMS),
EVEN IF SUCH HOLDER OR OTHER PARTY HAS BEEN ADVISED OF THE POSSIBILITY OF
SUCH DAMAGES.

  17. Interpretation of Sections 15 and 16.

  If the disclaimer of warranty and limitation of liability provided
above cannot be given local legal effect according to their terms,
reviewing courts shall apply local law that most closely approximates
an absolute waiver of all civil liability in connection with the
Program, unless a warranty or assumption of liability accompanies a
copy of the Program in return for a fee.

                     END OF TERMS AND CONDITIONS

            How to Apply These Terms to Your New Programs

  If you develop a new program, and you want it to be of the greatest
possible use to the public, the best way to achieve this is to make it
free software which everyone can redistribute and change under these terms.

  To do so, attach the following notices to the program.  It is safest
to attach them to the start of each source file to most effectively
state the exclusion of warranty; and each file should have at least
the "copyright" line and a pointer to where the full notice is found.

    <one line to give the program's name and a brief idea of what it does.>
    Copyright (C) <year>  <name of author>

    This program is free software: you can redistribute it and/or modify
    it under the terms of the GNU General Public License as published by
    the Free Software Foundation, either version 3 of the License, or
    (at your option) any later version.

    This program is distributed in the hope that it will be useful,
    but WITHOUT ANY WARRANTY; without even the implied warranty of
    MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
    GNU General Public License for more details.

    You should have received a copy of the GNU General Public License
    along with this program.  If not, see <https://www.gnu.org/licenses/>.

Also add information on how to contact you by electronic and paper mail.

  If the program does terminal interaction, make it output a short
notice like this when it starts in an interactive mode:

    <program>  Copyright (C) <year>  <name of author>
    This program comes with ABSOLUTELY NO WARRANTY; for details type `show w'.
    This is free software, and you are welcome to redistribute it
    under certain conditions; type `show c' for details.

The hypothetical commands `show w' and `show c' should show the appropriate
parts of the General Public License.  Of course, your program's commands
might be different; for a GUI interface, you would use an "about box".

  You should also get your employer (if you work as a programmer) or school,
if any, to sign a "copyright disclaimer" for the program, if necessary.
For more information on this, and how to apply and follow the GNU GPL, see
<https://www.gnu.org/licenses/>.

  The GNU General Public License does not permit incorporating your program
into proprietary programs.  If your program is a subroutine library, you
may consider it more useful to permit linking proprietary applications with
the library.  If this is what you want to do, use the GNU Lesser General
Public License instead of this License.  But first, please read
<https://www.gnu.org/licenses/why-not-lgpl.html>.

##GOOGLE RESEARCH - FRAME INTERPOLATION
Apache License
                           Version 2.0, January 2004
                        http://www.apache.org/licenses/

   TERMS AND CONDITIONS FOR USE, REPRODUCTION, AND DISTRIBUTION

   1. Definitions.

      "License" shall mean the terms and conditions for use, reproduction,
      and distribution as defined by Sections 1 through 9 of this document.

      "Licensor" shall mean the copyright owner or entity authorized by
      the copyright owner that is granting the License.

      "Legal Entity" shall mean the union of the acting entity and all
      other entities that control, are controlled by, or are under common
      control with that entity. For the purposes of this definition,
      "control" means (i) the power, direct or indirect, to cause the
      direction or management of such entity, whether by contract or
      otherwise, or (ii) ownership of fifty percent (50%) or more of the
      outstanding shares, or (iii) beneficial ownership of such entity.

      "You" (or "Your") shall mean an individual or Legal Entity
      exercising permissions granted by this License.

      "Source" form shall mean the preferred form for making modifications,
      including but not limited to software source code, documentation
      source, and configuration files.

      "Object" form shall mean any form resulting from mechanical
      transformation or translation of a Source form, including but
      not limited to compiled object code, generated documentation,
      and conversions to other media types.

      "Work" shall mean the work of authorship, whether in Source or
      Object form, made available under the License, as indicated by a
      copyright notice that is included in or attached to the work
      (an example is provided in the Appendix below).

      "Derivative Works" shall mean any work, whether in Source or Object
      form, that is based on (or derived from) the Work and for which the
      editorial revisions, annotations, elaborations, or other modifications
      represent, as a whole, an original work of authorship. For the purposes
      of this License, Derivative Works shall not include works that remain
      separable from, or merely link (or bind by name) to the interfaces of,
      the Work and Derivative Works thereof.

      "Contribution" shall mean any work of authorship, including
      the original version of the Work and any modifications or additions
      to that Work or Derivative Works thereof, that is intentionally
      submitted to Licensor for inclusion in the Work by the copyright owner
      or by an individual or Legal Entity authorized to submit on behalf of
      the copyright owner. For the purposes of this definition, "submitted"
      means any form of electronic, verbal, or written communication sent
      to the Licensor or its representatives, including but not limited to
      communication on electronic mailing lists, source code control systems,
      and issue tracking systems that are managed by, or on behalf of, the
      Licensor for the purpose of discussing and improving the Work, but
      excluding communication that is conspicuously marked or otherwise
      designated in writing by the copyright owner as "Not a Contribution."

      "Contributor" shall mean Licensor and any individual or Legal Entity
      on behalf of whom a Contribution has been received by Licensor and
      subsequently incorporated within the Work.

   2. Grant of Copyright License. Subject to the terms and conditions of
      this License, each Contributor hereby grants to You a perpetual,
      worldwide, non-exclusive, no-charge, royalty-free, irrevocable
      copyright license to reproduce, prepare Derivative Works of,
      publicly display, publicly perform, sublicense, and distribute the
      Work and such Derivative Works in Source or Object form.

   3. Grant of Patent License. Subject to the terms and conditions of
      this License, each Contributor hereby grants to You a perpetual,
      worldwide, non-exclusive, no-charge, royalty-free, irrevocable
      (except as stated in this section) patent license to make, have made,
      use, offer to sell, sell, import, and otherwise transfer the Work,
      where such license applies only to those patent claims licensable
      by such Contributor that are necessarily infringed by their
      Contribution(s) alone or by combination of their Contribution(s)
      with the Work to which such Contribution(s) was submitted. If You
      institute patent litigation against any entity (including a
      cross-claim or counterclaim in a lawsuit) alleging that the Work
      or a Contribution incorporated within the Work constitutes direct
      or contributory patent infringement, then any patent licenses
      granted to You under this License for that Work shall terminate
      as of the date such litigation is filed.

   4. Redistribution. You may reproduce and distribute copies of the
      Work or Derivative Works thereof in any medium, with or without
      modifications, and in Source or Object form, provided that You
      meet the following conditions:

      (a) You must give any other recipients of the Work or
          Derivative Works a copy of this License; and

      (b) You must cause any modified files to carry prominent notices
          stating that You changed the files; and

      (c) You must retain, in the Source form of any Derivative Works
          that You distribute, all copyright, patent, trademark, and
          attribution notices from the Source form of the Work,
          excluding those notices that do not pertain to any part of
          the Derivative Works; and

      (d) If the Work includes a "NOTICE" text file as part of its
          distribution, then any Derivative Works that You distribute must
          include a readable copy of the attribution notices contained
          within such NOTICE file, excluding those notices that do not
          pertain to any part of the Derivative Works, in at least one
          of the following places: within a NOTICE text file distributed
          as part of the Derivative Works; within the Source form or
          documentation, if provided along with the Derivative Works; or,
          within a display generated by the Derivative Works, if and
          wherever such third-party notices normally appear. The contents
          of the NOTICE file are for informational purposes only and
          do not modify the License. You may add Your own attribution
          notices within Derivative Works that You distribute, alongside
          or as an addendum to the NOTICE text from the Work, provided
          that such additional attribution notices cannot be construed
          as modifying the License.

      You may add Your own copyright statement to Your modifications and
      may provide additional or different license terms and conditions
      for use, reproduction, or distribution of Your modifications, or
      for any such Derivative Works as a whole, provided Your use,
      reproduction, and distribution of the Work otherwise complies with
      the conditions stated in this License.

   5. Submission of Contributions. Unless You explicitly state otherwise,
      any Contribution intentionally submitted for inclusion in the Work
      by You to the Licensor shall be under the terms and conditions of
      this License, without any additional terms or conditions.
      Notwithstanding the above, nothing herein shall supersede or modify
      the terms of any separate license agreement you may have executed
      with Licensor regarding such Contributions.

   6. Trademarks. This License does not grant permission to use the trade
      names, trademarks, service marks, or product names of the Licensor,
      except as required for reasonable and customary use in describing the
      origin of the Work and reproducing the content of the NOTICE file.

   7. Disclaimer of Warranty. Unless required by applicable law or
      agreed to in writing, Licensor provides the Work (and each
      Contributor provides its Contributions) on an "AS IS" BASIS,
      WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or
      implied, including, without limitation, any warranties or conditions
      of TITLE, NON-INFRINGEMENT, MERCHANTABILITY, or FITNESS FOR A
      PARTICULAR PURPOSE. You are solely responsible for determining the
      appropriateness of using or redistributing the Work and assume any
      risks associated with Your exercise of permissions under this License.

   8. Limitation of Liability. In no event and under no legal theory,
      whether in tort (including negligence), contract, or otherwise,
      unless required by applicable law (such as deliberate and grossly
      negligent acts) or agreed to in writing, shall any Contributor be
      liable to You for damages, including any direct, indirect, special,
      incidental, or consequential damages of any character arising as a
      result of this License or out of the use or inability to use the
      Work (including but not limited to damages for loss of goodwill,
      work stoppage, computer failure or malfunction, or any and all
      other commercial damages or losses), even if such Contributor
      has been advised of the possibility of such damages.

   9. Accepting Warranty or Additional Liability. While redistributing
      the Work or Derivative Works thereof, You may choose to offer,
      and charge a fee for, acceptance of support, warranty, indemnity,
      or other liability obligations and/or rights consistent with this
      License. However, in accepting such obligations, You may act only
      on Your own behalf and on Your sole responsibility, not on behalf
      of any other Contributor, and only if You agree to indemnify,
      defend, and hold each Contributor harmless for any liability
      incurred by, or claims asserted against, such Contributor by reason
      of your accepting any such warranty or additional liability.

   END OF TERMS AND CONDITIONS

   APPENDIX: How to apply the Apache License to your work.

      To apply the Apache License to your work, attach the following
      boilerplate notice, with the fields enclosed by brackets "[]"
      replaced with your own identifying information. (Don't include
      the brackets!)  The text should be enclosed in the appropriate
      comment syntax for the file format. We also recommend that a
      file or class name and description of purpose be included on the
      same "printed page" as the copyright notice for easier
      identification within third-party archives.

   Copyright [yyyy] [name of copyright owner]

   Licensed under the Apache License, Version 2.0 (the "License");
   you may not use this file except in compliance with the License.
   You may obtain a copy of the License at

       http://www.apache.org/licenses/LICENSE-2.0

   Unless required by applicable law or agreed to in writing, software
   distributed under the License is distributed on an "AS IS" BASIS,
   WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
   See the License for the specific language governing permissions and
   limitations under the License.

##ESRGAN

Apache License
                           Version 2.0, January 2004
                        http://www.apache.org/licenses/

   TERMS AND CONDITIONS FOR USE, REPRODUCTION, AND DISTRIBUTION

   1. Definitions.

      "License" shall mean the terms and conditions for use, reproduction,
      and distribution as defined by Sections 1 through 9 of this document.

      "Licensor" shall mean the copyright owner or entity authorized by
      the copyright owner that is granting the License.

      "Legal Entity" shall mean the union of the acting entity and all
      other entities that control, are controlled by, or are under common
      control with that entity. For the purposes of this definition,
      "control" means (i) the power, direct or indirect, to cause the
      direction or management of such entity, whether by contract or
      otherwise, or (ii) ownership of fifty percent (50%) or more of the
      outstanding shares, or (iii) beneficial ownership of such entity.

      "You" (or "Your") shall mean an individual or Legal Entity
      exercising permissions granted by this License.

      "Source" form shall mean the preferred form for making modifications,
      including but not limited to software source code, documentation
      source, and configuration files.

      "Object" form shall mean any form resulting from mechanical
      transformation or translation of a Source form, including but
      not limited to compiled object code, generated documentation,
      and conversions to other media types.

      "Work" shall mean the work of authorship, whether in Source or
      Object form, made available under the License, as indicated by a
      copyright notice that is included in or attached to the work
      (an example is provided in the Appendix below).

      "Derivative Works" shall mean any work, whether in Source or Object
      form, that is based on (or derived from) the Work and for which the
      editorial revisions, annotations, elaborations, or other modifications
      represent, as a whole, an original work of authorship. For the purposes
      of this License, Derivative Works shall not include works that remain
      separable from, or merely link (or bind by name) to the interfaces of,
      the Work and Derivative Works thereof.

      "Contribution" shall mean any work of authorship, including
      the original version of the Work and any modifications or additions
      to that Work or Derivative Works thereof, that is intentionally
      submitted to Licensor for inclusion in the Work by the copyright owner
      or by an individual or Legal Entity authorized to submit on behalf of
      the copyright owner. For the purposes of this definition, "submitted"
      means any form of electronic, verbal, or written communication sent
      to the Licensor or its representatives, including but not limited to
      communication on electronic mailing lists, source code control systems,
      and issue tracking systems that are managed by, or on behalf of, the
      Licensor for the purpose of discussing and improving the Work, but
      excluding communication that is conspicuously marked or otherwise
      designated in writing by the copyright owner as "Not a Contribution."

      "Contributor" shall mean Licensor and any individual or Legal Entity
      on behalf of whom a Contribution has been received by Licensor and
      subsequently incorporated within the Work.

   2. Grant of Copyright License. Subject to the terms and conditions of
      this License, each Contributor hereby grants to You a perpetual,
      worldwide, non-exclusive, no-charge, royalty-free, irrevocable
      copyright license to reproduce, prepare Derivative Works of,
      publicly display, publicly perform, sublicense, and distribute the
      Work and such Derivative Works in Source or Object form.

   3. Grant of Patent License. Subject to the terms and conditions of
      this License, each Contributor hereby grants to You a perpetual,
      worldwide, non-exclusive, no-charge, royalty-free, irrevocable
      (except as stated in this section) patent license to make, have made,
      use, offer to sell, sell, import, and otherwise transfer the Work,
      where such license applies only to those patent claims licensable
      by such Contributor that are necessarily infringed by their
      Contribution(s) alone or by combination of their Contribution(s)
      with the Work to which such Contribution(s) was submitted. If You
      institute patent litigation against any entity (including a
      cross-claim or counterclaim in a lawsuit) alleging that the Work
      or a Contribution incorporated within the Work constitutes direct
      or contributory patent infringement, then any patent licenses
      granted to You under this License for that Work shall terminate
      as of the date such litigation is filed.

   4. Redistribution. You may reproduce and distribute copies of the
      Work or Derivative Works thereof in any medium, with or without
      modifications, and in Source or Object form, provided that You
      meet the following conditions:

      (a) You must give any other recipients of the Work or
          Derivative Works a copy of this License; and

      (b) You must cause any modified files to carry prominent notices
          stating that You changed the files; and

      (c) You must retain, in the Source form of any Derivative Works
          that You distribute, all copyright, patent, trademark, and
          attribution notices from the Source form of the Work,
          excluding those notices that do not pertain to any part of
          the Derivative Works; and

      (d) If the Work includes a "NOTICE" text file as part of its
          distribution, then any Derivative Works that You distribute must
          include a readable copy of the attribution notices contained
          within such NOTICE file, excluding those notices that do not
          pertain to any part of the Derivative Works, in at least one
          of the following places: within a NOTICE text file distributed
          as part of the Derivative Works; within the Source form or
          documentation, if provided along with the Derivative Works; or,
          within a display generated by the Derivative Works, if and
          wherever such third-party notices normally appear. The contents
          of the NOTICE file are for informational purposes only and
          do not modify the License. You may add Your own attribution
          notices within Derivative Works that You distribute, alongside
          or as an addendum to the NOTICE text from the Work, provided
          that such additional attribution notices cannot be construed
          as modifying the License.

      You may add Your own copyright statement to Your modifications and
      may provide additional or different license terms and conditions
      for use, reproduction, or distribution of Your modifications, or
      for any such Derivative Works as a whole, provided Your use,
      reproduction, and distribution of the Work otherwise complies with
      the conditions stated in this License.

   5. Submission of Contributions. Unless You explicitly state otherwise,
      any Contribution intentionally submitted for inclusion in the Work
      by You to the Licensor shall be under the terms and conditions of
      this License, without any additional terms or conditions.
      Notwithstanding the above, nothing herein shall supersede or modify
      the terms of any separate license agreement you may have executed
      with Licensor regarding such Contributions.

   6. Trademarks. This License does not grant permission to use the trade
      names, trademarks, service marks, or product names of the Licensor,
      except as required for reasonable and customary use in describing the
      origin of the Work and reproducing the content of the NOTICE file.

   7. Disclaimer of Warranty. Unless required by applicable law or
      agreed to in writing, Licensor provides the Work (and each
      Contributor provides its Contributions) on an "AS IS" BASIS,
      WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or
      implied, including, without limitation, any warranties or conditions
      of TITLE, NON-INFRINGEMENT, MERCHANTABILITY, or FITNESS FOR A
      PARTICULAR PURPOSE. You are solely responsible for determining the
      appropriateness of using or redistributing the Work and assume any
      risks associated with Your exercise of permissions under this License.

   8. Limitation of Liability. In no event and under no legal theory,
      whether in tort (including negligence), contract, or otherwise,
      unless required by applicable law (such as deliberate and grossly
      negligent acts) or agreed to in writing, shall any Contributor be
      liable to You for damages, including any direct, indirect, special,
      incidental, or consequential damages of any character arising as a
      result of this License or out of the use or inability to use the
      Work (including but not limited to damages for loss of goodwill,
      work stoppage, computer failure or malfunction, or any and all
      other commercial damages or losses), even if such Contributor
      has been advised of the possibility of such damages.

   9. Accepting Warranty or Additional Liability. While redistributing
      the Work or Derivative Works thereof, You may choose to offer,
      and charge a fee for, acceptance of support, warranty, indemnity,
      or other liability obligations and/or rights consistent with this
      License. However, in accepting such obligations, You may act only
      on Your own behalf and on Your sole responsibility, not on behalf
      of any other Contributor, and only if You agree to indemnify,
      defend, and hold each Contributor harmless for any liability
      incurred by, or claims asserted against, such Contributor by reason
      of your accepting any such warranty or additional liability.

   END OF TERMS AND CONDITIONS

   APPENDIX: How to apply the Apache License to your work.

      To apply the Apache License to your work, attach the following
      boilerplate notice, with the fields enclosed by brackets "[]"
      replaced with your own identifying information. (Don't include
      the brackets!)  The text should be enclosed in the appropriate
      comment syntax for the file format. We also recommend that a
      file or class name and description of purpose be included on the
      same "printed page" as the copyright notice for easier
      identification within third-party archives.

   Copyright [yyyy] [name of copyright owner]

   Licensed under the Apache License, Version 2.0 (the "License");
   you may not use this file except in compliance with the License.
   You may obtain a copy of the License at

       http://www.apache.org/licenses/LICENSE-2.0

   Unless required by applicable law or agreed to in writing, software
   distributed under the License is distributed on an "AS IS" BASIS,
   WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
   See the License for the specific language governing permissions and
   limitations under the License.